# Case 2 - Siemens Advanta

Group W


## 1. Loading the Data

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import plotly.express as px


from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.stats.outliers_influence import variance_inflation_factor

import folium
import requests
import json
from branca.colormap import linear

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import  mean_squared_error, root_mean_squared_error

from prophet import Prophet 
from darts import TimeSeries
from darts.models import XGBModel
from darts.metrics import rmse
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from darts.models.forecasting.xgboost import XGBModel

from sklearn.ensemble import RandomForestRegressor

#%pip install nixtla
from nixtla import NixtlaClient

#%pip install neuralforecast
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS

import statsmodels.tsa.stattools as ts

import warnings
warnings.filterwarnings('ignore')

The group started by loading the two data files, `Case2_Market data.xlsx` and `Case2_Sales data.csv`.

In [ ]:
sales_db = pd.read_csv("datasets\Case2_Sales data.csv", sep=";")
market_db = pd.read_excel("datasets\Case2_Market data.xlsx")

## 2. Data Understanding

### Sales Data
The group decided to take a look a `sales_db` first.

In [ ]:
sales_db

In [ ]:
sales_db.info()

`Sales_EUR` was composed of numeric values yet it was stored as an object. There was also an error in `DATE` which is stored as a object instead of a date. These were the first errors to address so that EDA could be performed.

In [ ]:
sales_db['Sales_EUR'] = sales_db['Sales_EUR'].str.replace(',', '.').astype(float)
sales_db['DATE'] = pd.to_datetime(sales_db['DATE'], format='%d.%m.%Y')

In [ ]:
# # export data to csv (PowerBI use)
# sales_db.to_csv('sales_overview.csv', decimal=',')

In [ ]:
sales_db.describe()

Using the describe method the group found that there was at least one negative value in the `Sales_EUR` column. The group then decided to check for more negative data entries.

In [ ]:
negative_sales = sales_db[sales_db['Sales_EUR'] < 0]
negative_sales

There were more than 200 rows with negative sales values, but were there any underlying patterns that could explain these anomalies?

In [ ]:
negative_sales['DATE'].dt.year.value_counts()

In [ ]:
negative_sales['DATE'].dt.month.value_counts()

In [ ]:
negative_sales['DATE'].dt.day.value_counts()

The negative values happened at random times, but could they possibly have been connected to a single `Mapped_GCK` (or a group of GCK's)?

In [ ]:
negative_sales['Mapped_GCK'].value_counts()

Much like the dates, it seemed random. The group concluded that there likely wasn't any explanation. Later at the Q&A session with Siemens Advanta the group was informed that these negative values were actually returns.

### Market Data

In [ ]:
market_db

First, the group decided to fix the column names so that only one index existed. This was done by joining the country names at the end (or beginning) of the column name.

In [ ]:
new_col_names = ['DATE',
                'Production Index Machinery & Electricals China', 'Shipments Index Machinery & Electricals China',
                'Production Index Machinery & Electricals France', 'Shipments Index Machinery & Electricals France',
                'Production Index Machinery & Electricals Germany', 'Shipments Index Machinery & Electricals Germany',
                'Production Index Machinery & Electricals Italy', 'Shipments Index Machinery & Electricals Italy',
                'Production Index Machinery & Electricals Japan', 'Shipments Index Machinery & Electricals Japan',
                'Production Index Machinery & Electricals Switzerland', 'Shipments Index Machinery & Electricals Switzerland',
                'Production Index Machinery & Electricals United Kingdom', 'Shipments Index Machinery & Electricals United Kingdom',
                'Production Index Machinery & Electricals United States', 'Shipments Index Machinery & Electricals United States',
                'Production Index Machinery & Electricals Europe', 'Shipments Index Machinery & Electricals Europe',
                'World: Price of Base Metals', 'World: Price of Energy', 'World: Price of Metals & Minerals', 'World: Price of Natural gas index', 'World: Price of Crude oil, average', 'World: Price of Copper',
                'United States: EUR in LCU',
                'Producer Prices United States: Electrical equipment',
                'Producer Prices United Kingdom: Electrical equipment',
                'Producer Prices Italy: Electrical equipment',
                'Producer Prices France: Electrical equipment',
                'Producer Prices Germany: Electrical equipment',
                'Producer Prices China: Electrical equipment',
                'Production Index United States: Machinery and equipment n.e.c.',
                'Prodcution Index World: Machinery and equipment n.e.c.',
                'Production Index Switzerland: Machinery and equipment n.e.c.',
                'Production Index United Kingdom: Machinery and equipment n.e.c.',
                'Production Index Italy: Machinery and equipment n.e.c.', 
                'Production Index  Japan: Machinery and equipment n.e.c.',
                'Production Index France: Machinery and equipment n.e.c.',
                'Production Index Germany: Machinery and equipment n.e.c.',
                'Production Index United States: Electrical equipment',
                'Production Index World: Electrical equipment',
                'Production Index Switzerland: Electrical equipment',
                'Production Index United Kingdom: Electrical equipment',
                'Production Index Italy: Electrical equipment',
                'Production Index Japan: Electrical equipment',
                'Production Index France: Electrical equipment','Production Index Germany: Electrical equipment']

market_db.columns = new_col_names
market_db.drop(0, inplace=True)

In [ ]:
market_db

With the new names in order the group decided to create a copy of the data where the codes such as "MAB_ELE_PRO156" where removed. This was done in a separate dataset because at this point the group had no knowledge of the relevance of these codes. To this point the group had theorized that these were probably codes that represented shortened versions of the column names. The codes were later deemed irrelevant for forecasting by Siemens in the Q&A session.

In [ ]:
market_db.drop(1, inplace=True)
market_db.reset_index(drop=True, inplace=True)
market_db.head()

In [ ]:
for col in market_db.columns[1:48]:
    market_db[col] = pd.to_numeric(market_db[col], errors='coerce')
market_db.dtypes

In [ ]:
market_db1 = market_db.copy()
market_db1

In [ ]:
market_db1['DATE'] = pd.to_datetime(market_db1['DATE'].str.strip().str.replace('m', '-'), format='%Y-%m')
market_db1

In [ ]:
# # export data to csv (for PowerBI use)
# market_db1.to_csv('market_indexes.csv', decimal=',')

In [ ]:
nulls = market_db1.isnull().sum().sort_values(ascending=False)
nulls = nulls[nulls > 0]
nulls = pd.DataFrame(nulls, columns=['nulls'])
nulls['percentage'] = (nulls['nulls'] / len(market_db1) * 100).round(2)
nulls

Several columns had missing data, but the ones where the missing data percentage was the highest were:
- `Shipments Index Machinery & Electricals United Kingdom` (8%)
- `Producer Prices United Kingdom: Electrical equipment` (8%)
- `Producer Prices France: Electrical equipment` (15%)
- `Producer Prices China: Electrical equipment` (10%)
- `Production Index World: Electrical equipment` (5%).

The group decided to study these columns further to see wich dates had missing values.

In [ ]:
missing_dates = market_db1[market_db1.isnull().any(axis=1)]['DATE']
missing_years = missing_dates.dt.year
missing_years.value_counts()

Only a smaller portion of the missing rows happened in the time during which the available sales data occurs, as such the group decided to take a closer look at these entries.

In [ ]:
missing_dates_2018 = market_db1[(market_db1.isnull().any(axis=1)) | (market_db1['DATE'] > '2017-12-31')]['DATE']
missing_months = missing_dates_2018.dt.month
missing_months.value_counts()

There was at least a week of missing data in every month. Could this be the same week each time? This could possibly be explained by downtime when the server is being maintained or updated, thereby explaining the missing data.

In [ ]:
missing_dates_2018.sample(25)

Again there seems to be no particular reason for the missing data, as such it will be addressed in the data cleaning section. 

In [ ]:
market_db1.describe(). T

In [ ]:
market_db1.head()

The numbers reflect the value of that country's macroeconomic indicator for that month compared to the 2010 baseline:
- If the index was higher than 100, that indicator was performing above 2010 baseline
- If the index was below 100, was performing below the 2010 baseline

In [ ]:
market_db1["United States: EUR in LCU"].value_counts()

"LCU" means Local Currency Unit and since this column references the United States it indicates a conversion factor. For example, a value of 1.2770 means 1 EUR = 1.2770 USD at that month. 

Since the sales are in EUR but some macroeconomic indicators like commodity prices are quoted in USD, the group decided to convert them into the same currency by dividing it by the exchange rate. For example, if price of base metals was 50 USD in April 2020 and the exchange rate is 1.10 USD/EUR, then the price in Eur = 50 / 1.10 = 45.45 EUR.

In [ ]:
price_columns = [
    'World: Price of Base Metals', 
    'World: Price of Energy', 
    'World: Price of Metals & Minerals',
    'World: Price of Natural gas index', 
    'World: Price of Crude oil, average', 
    'World: Price of Copper',
    'Producer Prices United States: Electrical equipment', 
    'Producer Prices United Kingdom: Electrical equipment',
    'Producer Prices Italy: Electrical equipment', 
    'Producer Prices France: Electrical equipment',
    'Producer Prices Germany: Electrical equipment', 
    'Producer Prices China: Electrical equipment'
]

for col in price_columns:
    market_db1[col] = market_db1[col] / market_db1['United States: EUR in LCU']

market_db1.head()

In [ ]:
describe_df=market_db1.describe().T
describe_df[describe_df.duplicated(keep=False)]

Production Index Machinery & Electricals_China and Shipments Index Machinery & Electricals_China are the same.

## 3. Exploration and Data Analysis

### Sales Data

#### General View of the Sales Data

In [ ]:
sales_db['year'] = sales_db['DATE'].dt.year
sales_db['month'] = sales_db['DATE'].dt.month
sales_per_month = sales_db.groupby(['year', 'month'])['Sales_EUR'].sum().reset_index()

fig, ax = plt.subplots(figsize=(18, 4))
sns.set_style("whitegrid")
sns.lineplot(x='month', y='Sales_EUR', hue='year', data=sales_per_month, palette=['#003750', '#2387AA', '#E3700F', '#FFCD50', '#009999'])
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x / 1e6:.1f}M'))
sns.despine(left=True, bottom=True)
plt.title('Sales per month (2018-2020, in Millions of EUR)', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Sales (EUR)', fontsize=12)
plt.xticks(range(1, 13))
plt.show()

In [ ]:
sales_eda = sales_db.copy()

In [ ]:
sales_eda['week_day'] = sales_eda['DATE'].dt.day_name()
sales_per_weekday = sales_eda.groupby('week_day')['Sales_EUR'].sum().reset_index()
sales_per_weekday['week_day'] = pd.Categorical(sales_per_weekday['week_day'], categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)
sales_per_weekday = sales_per_weekday.sort_values('week_day')

fig, ax = plt.subplots(figsize=(10, 4))
sns.set_style("whitegrid")
sns.barplot(x='week_day', y='Sales_EUR', data=sales_per_weekday, palette=['#009999'])
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x / 1e6:.1f}M'))
sns.despine(left=True, bottom=True)
plt.title('Sales by Week Day (in Millions of EUR)', fontsize=16)
plt.xlabel('Week Day', fontsize=12)
plt.ylabel('Sales (EUR)', fontsize=12)
plt.show()

In [ ]:
# Calculate the percentage of sales grouped by weekdays
sales_count = sales_eda['week_day'].value_counts(normalize=True).reset_index()
sales_count.columns = ['week_day', 'percentage']
sales_count['percentage'] *= 100  # Convert to percentage
sales_count['week_day'] = pd.Categorical(
    sales_count['week_day'], 
    categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], 
    ordered=True
)
sales_count = sales_count.sort_values('week_day')

fig, ax = plt.subplots(figsize=(10, 4))
sns.set_style("whitegrid")
sns.barplot(x='week_day', y='percentage', data=sales_count, palette=['#009999'])
sns.despine(left=True, bottom=True)
plt.title('Sales Count by Week Day (Percentage)', fontsize=16)
plt.xlabel('Week Day', fontsize=12)
plt.ylabel('Percentage (%)', fontsize=12)
plt.show()

Looking at the sum of sales per month over the span of four years, some underlying patterns were identified.

- 2018 is the point where data collection started (in October), meaning it lacks a full year's representation.  
- January 2021 starts from a very low revenue point for Siemens (nearing 30 million euros), which could possibly be related to COVID-19.  
- 2021 deviates from the trends observed in other years.  
- 2022 had only four months of recorded data, showing particularly strong performance compared to other years.
- Sales consistently peak in September, which could be attributed to several factors:  
  - The end of the fiscal year (often in September or October) leads companies to spend their remaining budget to avoid cuts.  
  - Project deadlines drive increased purchasing.  
  - Inventory restocking before Q4 to prepare for year-end demand spikes.  
  - Seasonal demand fluctuations related to the industry.

> Note: This graph is a sum of all values per month in a given year as such it also considers negative values (returns).

After looking at how the sales behaved the group decided to see if there was any type of pattern or seasonality in the returns.

In [ ]:
negative_sales = sales_db[sales_db['Sales_EUR'] < 0]
negative_sales['year'] = negative_sales['DATE'].dt.year
negative_sales['month'] = negative_sales['DATE'].dt.month
sales_per_month = negative_sales.groupby(['year', 'month'])['Sales_EUR'].sum().reset_index()

fig, ax = plt.subplots(figsize=(18, 4))
sns.set_style("whitegrid")
sns.lineplot(x='month', y='Sales_EUR', hue='year', data=sales_per_month, palette=['#003750', '#2387AA', '#E3700F', '#FFCD50', '#009999'])
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x / 1e6:.1f}M'))
sns.despine(left=True, bottom=True)
plt.title('Returns per month (2018-2022, in Millions of EUR)', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Sales (EUR)', fontsize=12)
plt.xticks(range(1, 13))
plt.show()

When analysing the returns the group found that:
- 2019 (red line) shows relatively stable return rates through most of the year, hovering between -0.1M and -0.2M EUR, with a significant spike in returns around month 10.
- 2020 (pink line) shows dramatic improvement over the year, starting with very high returns (around -0.8M EUR) in January but recovering to near 0 by mid-year and maintaining that improvement.
- 2021 (purple line) displays a cyclical pattern with notable spikes in returns around months 4, 8, and 11.
- 2022 (dark purple line) shows volatility, beginning at -0.2M, improving to near 0 by month 3, declining again around month 4, with partial year data ending around mid-year.
- Seasonal patterns appear across years, with months 5-7 typically showing fewer returns, while months 3-4 and 10-11 often show higher return volumes.

What about the total value of returns per month?

In [ ]:
negative_sales['month'] = negative_sales['DATE'].dt.month

sales_per_month = negative_sales.groupby(['month'])['Sales_EUR'].sum().reset_index()
sales_per_month

fig, ax = plt.subplots(figsize=(18, 4))
sns.set_style('whitegrid')
sns.lineplot(x='month', y='Sales_EUR', data=sales_per_month, color='#009999')
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x / 1e6:.1f}M'))
sns.despine(bottom=True, left=True)
plt.title('Returns per month (all years, in Millions of EUR)', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Sales (EUR)', fontsize=12)
plt.xticks(range(1, 13))

When looking at the sum of returns for all years a couple of observations were made:
- January is the overall worst performing month with its loss surpassing 1 million euros.
- March and April feature the worst two month run with an overall loss of, just about, 2 million euros.
- May to August marks the most positive time of the year where returns are kept at about 200 thousand euro per month
- After August returns start to grow, a trend that only seems to stop in Januray (as previously mentioned).

The same overall analysis was performed for the general sales data.

In [ ]:
sales_db['month'] = sales_db['DATE'].dt.month

sales_per_month = sales_db.groupby(['month'])['Sales_EUR'].sum().reset_index()
sales_per_month

fig, ax = plt.subplots(figsize=(16, 6))
sns.set_style('whitegrid')
sns.lineplot(x='month', y='Sales_EUR', data=sales_per_month, color='#009999')
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x / 1e6:.1f}M'))
sns.despine(bottom=True, left=True)
plt.title('Sales per month (all years, in Millions of EUR)', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Sales (EUR)', fontsize=12)
plt.xticks(range(1, 13))

Heres the conclusions the group drew from this plot:
- Sales grow the most (and hit their peak) in the first 3 months of the year.
- The best selling month is March with over 260 million euros of sales.
- The worst performing month is May with just over 180 million euros of sales
- Much like returns, sales are at their lowest during the summer suggesting that this is an "off season" for these products where neither purchases nor returns are made.
- From Augsut to December sales start to rise taking a short fall in November (something that also happens in the returns).

> Note: This graph is a sum of all values per month in a given year as such it also considers negative values (returns).

#### Aggregating the data into months

Since the predictions were to be made on a monthly basis the group decided that it would be better to aggregate the daily sales into monthly sales. This meant that after the aggregation each row would correspond to the total sales value of a month for each GCK. If this value is negative it means that the overall returns on a given month are larger than the sales of that same month.

In [ ]:
sales_db['year'] = sales_db['DATE'].dt.year
sales_db['month'] = sales_db['DATE'].dt.month
sales_db = sales_db.groupby(['year', 'month', 'Mapped_GCK'], as_index=False).agg({'Sales_EUR': 'sum', 'DATE': 'first'})
sales_db = sales_db[['DATE', 'Mapped_GCK', 'Sales_EUR']]
sales_db


With the monthly data, the group decided to look for any specificities in the mapped GCK's.

In [ ]:
highlighted_gcks = {'#1': '#003750', '#3': '#E3700F', '#5': '#009999'}
palette = {gck: highlighted_gcks[gck] if gck in highlighted_gcks else 'gray' for gck in sales_db['Mapped_GCK'].unique()}

fig, ax = plt.subplots(figsize=(14, 6))
sns.set_style('whitegrid')
sns.lineplot(x='DATE', y='Sales_EUR', hue='Mapped_GCK', data=sales_db, palette=palette)

handles, labels = ax.get_legend_handles_labels()
filtered_handles_labels = [(h, l) for h, l in zip(handles, labels) if l in highlighted_gcks]
handles, labels = zip(*filtered_handles_labels)
ax.legend(handles, labels, title='GCK')

ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x / 1e6:.1f}M'))
sns.despine(bottom=True, left=True)
plt.title('Sales per GCK (2018-2022, in Millions of EUR)', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Sales (EUR)', fontsize=12)
plt.xticks(rotation=45)

From the line chart above the group concluded that there are 3 GKC's that stand out from the rest. GCK *#3* and *#5* manage to create some distance from the main group (ilustrated in grey) while keeping close to each other, but GCK *#1* represents the product group with the most sales more than doubling the sales of *#3* or *#5*.

In [ ]:
negative_sales = sales_db[sales_db['Sales_EUR'] < 0]
negative_sales = negative_sales.sort_values('Sales_EUR')
negative_sales

Regarding "negative sales" the group found that GCK's *#6*, *#9*, *#14* are the only products where the sales have a negative value. Note that this values are *post-agregation*, meaning these are the only four months where the returns "out-performed" the sales.

Noting the significant differences in sales scales across GCKs, the group decided to categorize them into low, medium, and high sales. Two of the GCKs (*#9* and *#20*) had to be analyzed separately due to their scale.

> Note: These groupings were used solely for visualization purposes.

In [ ]:
high_sales = ['#1', '#3', '#5']
low_sales = ['#13', '#14', '#36']
mid_sales = ['#16', '#11', '#12', '#6', '#8']

sales_db['month'] = sales_db['DATE'].dt.month

#### High Sales

In [ ]:
highlighted_gcks = {'#1': '#003750', '#3': '#E3700F', '#5': '#009999'}
group_data = sales_db[sales_db['Mapped_GCK'].isin(high_sales)]
monthly_avg = group_data.groupby('month')['Sales_EUR'].mean().reset_index()
group_name = 'High Sales'

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(10, 5))
for gck in high_sales:
    gck_data = group_data[group_data['Mapped_GCK'] == gck]
    monthly_avg_gck = gck_data.groupby('month')['Sales_EUR'].mean().reset_index()
    sns.lineplot(data=monthly_avg_gck, x='month', y='Sales_EUR', label=f'{gck}', color=highlighted_gcks[gck])
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x / 1e6:.1f}M'))
plt.title(f'Monthly Average Sales for {group_name} GCKs', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Average Sales (EUR)', fontsize=12)
plt.xticks(range(1, 13))
sns.despine(bottom=True, left=True)
plt.legend()
plt.show()


Firstly, within the high-sales group, GCK #1 consistently outperformed the others, maintaining average monthly sales above 30 million euros. GCKs #3 and #5 exhibited similar patterns, with both experiencing growth and stabilization in the first trimester, followed by an increase and subsequent decline between August and October. Among them, GCK #5 had the lowest sales, at times barely exceeding 5 million euros.

#### Medium Sales

In [ ]:
highlighted_gcks = {'#16': '#003750', '#11': '#2387AA', '#8': '#E3700F', '#12': '#FFCD50', '#6': '#009999'}
group_data = sales_db[sales_db['Mapped_GCK'].isin(mid_sales)]
monthly_avg = group_data.groupby('month')['Sales_EUR'].mean().reset_index()
group_name = 'Medium Sales'

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(10, 5))
for gck in mid_sales:
    gck_data = group_data[group_data['Mapped_GCK'] == gck]
    monthly_avg_gck = gck_data.groupby('month')['Sales_EUR'].mean().reset_index()
    sns.lineplot(data=monthly_avg_gck, x='month', y='Sales_EUR', label=f'{gck}',  color=highlighted_gcks[gck])
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x / 1e6:.1f}M'))
plt.title(f'Monthly Average Sales for {group_name} GCKs', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Average Sales (EUR)', fontsize=12)
plt.xticks(range(1, 13))
sns.despine(bottom=True, left=True)
plt.legend()
plt.show()

The medium-sales group exhibits a wide range, with sales fluctuating from just a few thousand euros to over 3.5 million. GCK #11 is the highest-selling on average, displaying two distinct peaks in March (which also happens for *#16* and *#8*) and September—the latter also observed in GCK #8. All GCKs in this group experienced an uptick in sales during the final trimester of the year. The weakest performer, GCK #12, never surpassed half a million in sales, with its best months occurring in September and November.

#### Low Sales

In [ ]:
highlighted_gcks = {'#13': '#003750', '#14': '#E3700F', '#36': '#009999'}
group_data = sales_db[sales_db['Mapped_GCK'].isin(low_sales)]
monthly_avg = group_data.groupby('month')['Sales_EUR'].mean().reset_index()
group_name = 'Low Sales'

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(10, 5))
for gck in low_sales:
    gck_data = group_data[group_data['Mapped_GCK'] == gck]
    monthly_avg_gck = gck_data.groupby('month')['Sales_EUR'].mean().reset_index()
    sns.lineplot(data=monthly_avg_gck, x='month', y='Sales_EUR', label=f'{gck}', color=highlighted_gcks[gck])
plt.title(f'Monthly Average Sales for {group_name} GCKs', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Average Sales (EUR)', fontsize=12)
plt.xticks(range(1, 13))
sns.despine(bottom=True, left=True)
plt.legend()
plt.show()


The low-sales group rarely exceeds 70,000 euros, with only GCK #36 reaching this level in two months (April and June). All other values in the group remain below 45,000 euros. GCK #36 exhibits highly volatile sales, in contrast to GCK #13, which maintains relatively stable sales throughout the year, except for a dip in May. GCK #14 experiences its strongest period from April to July, rising from nearly no sales to over 35,000 euros.

#### GCKs #9 and #20

In [ ]:
group_data = sales_db[sales_db['Mapped_GCK'] == '#9']
monthly_avg = group_data.groupby('month')['Sales_EUR'].mean().reset_index()
group_name = 'GCK #9'

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(data=monthly_avg, x='month', y='Sales_EUR', color = '#009999')
plt.title(f'Monthly Average Sales for {group_name}', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Average Sales (EUR)', fontsize=12)
plt.xticks(range(1, 13))
sns.despine(bottom=True, left=True)
plt.show()

GCK *#9* shows a sharp spike in March, hitting just over 12,000 euros, before dropping back down in the following months. There's some fluctuation mid-year, but the trend is mostly downward until August. From there, sales climb steadily, peaking again in October at just above 11,000 euros before tapering off in the last two months. A volatile pattern overall, with two clear surges in March and October.

In [ ]:
group_data = sales_db[sales_db['Mapped_GCK'] == '#20']
monthly_avg = group_data.groupby('month')['Sales_EUR'].mean().reset_index()
group_name = 'GCK #9'

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(data=monthly_avg, x='month', y='Sales_EUR', color = '#009999')
plt.title(f'Monthly Average Sales for {group_name}', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Average Sales (EUR)', fontsize=12)
plt.xticks(range(1, 13))
sns.despine(bottom=True, left=True)
plt.show()

GCK *#20* has a massive spike in April, reaching over 4,500 euros, but immediately crashes in May, barely clearing 500 euros. Before that, sales are mostly flat, staying under 1,000 euros. After the drop, there's some minor fluctuation mid-year, but nothing substantial. Then, another surge in October, peaking just above 3,000 euros, followed by a gradual decline. A pattern of two big spikes with long periods of low activity in between.

#### Final Conclusions

The group was able to conclude that on average:

- There are two peaks during the year happening just before summer season and right afterwards
- During the summer sales seem to drop and stabilize
- In some cases sales seem to rise during the last trimester, although this seems to be a trend that mainly affects the medium sales group

In [ ]:
sales_db.drop(columns=['month'], inplace=True)

#### Checking for characteritics and patterns

Before going into the modelling phase, the group decomposed the time series to extract the Seasonality, Trends, Autocorrelation and Stationarity.

In [ ]:
sales_product = {}
mapped_gck_list = sales_db['Mapped_GCK'].unique().tolist()
for i in mapped_gck_list:
    sales_product[i] = sales_db[sales_db["Mapped_GCK"] == i].set_index("DATE")
    sales_product[i].drop(columns=["Mapped_GCK"], inplace=True)

##### Seasonality and Trends

In [ ]:
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#009999'])
for i in mapped_gck_list:
    result = seasonal_decompose(sales_product[i], model='additive', period=12)
    fig = result.plot()
    fig.set_size_inches(7, 4)
    plt.suptitle(f'Seasonal Decomposition for GCK {i}', y=1.02)
    plt.show()

Regarding trends the product groups were distributed as follows:
- Uptrend: #11; #12; #36; #8
- Horizontal Trend: #1; #14; #20; #3; #4; #5; #9
- Downtrend: #13; #16; #6

The majority of graphics appeared to have some type of seasonality (mostly visble from the third image onwards).

##### Autocorrelation (ACF and PACF)

**ACF**

In [ ]:
for i in mapped_gck_list:
    fig = plot_acf(sales_product[i], title=f'Autocorrelation for GCK {i}')
    plt.show()

The data did not seem to be autocorrelated.

**PACF**

In [ ]:
for i in mapped_gck_list:
    fig = plot_pacf(sales_product[i], title=f'Autocorrelation for GCK {i}')
    plt.show()

Based on the analysis of the ACF and PAFC plots, most of the product groups do not show meaningful correlations. This means that past values only have a minor influence on future values. Adding external data (market_db) could possibly enhance the predictive power.

##### Stationarity (Dickey-Fuller test)

In [ ]:
for i in mapped_gck_list:
    adf, pval, usedlag, nobs, crit_vals, icbest = adfuller(sales_product[i]["Sales_EUR"].values)
    print(f'GCK {i}')
    print('ADF test statistic:', adf)
    print('ADF p-value:', pval)
    print('Number of lags used:', usedlag)
    print('Number of observations:', nobs)
    print('Critical values:', crit_vals)
    print('Best information criterion:', icbest, '\n')

All product groups except one were stationary. Product group #8 isn't and therefore it will need to undergo a transformation (differencing) to become stationary.

### Market Data

In [ ]:
production_index_machinery_eletricals = ['Production Index Machinery & Electricals China', 
                                         'Production Index Machinery & Electricals France',
                                         'Production Index Machinery & Electricals Germany', 
                                         'Production Index Machinery & Electricals Italy',
                                         'Production Index Machinery & Electricals Japan',
                                         'Production Index Machinery & Electricals Switzerland',
                                         'Production Index Machinery & Electricals United Kingdom', 
                                         'Production Index Machinery & Electricals United States',
                                         'Production Index Machinery & Electricals Europe']
                    
production_index_machinery = ['Production Index United States: Machinery and equipment n.e.c.',
                            'Prodcution Index World: Machinery and equipment n.e.c.',
                            'Production Index Switzerland: Machinery and equipment n.e.c.',
                            'Production Index United Kingdom: Machinery and equipment n.e.c.',
                            'Production Index Italy: Machinery and equipment n.e.c.',
                            'Production Index  Japan: Machinery and equipment n.e.c.',
                            'Production Index France: Machinery and equipment n.e.c.',
                            'Production Index Germany: Machinery and equipment n.e.c.']

production_index_eletrical = ['Production Index United States: Electrical equipment',
                            'Production Index World: Electrical equipment',
                            'Production Index Switzerland: Electrical equipment',
                            'Production Index United Kingdom: Electrical equipment',
                            'Production Index Italy: Electrical equipment',
                            'Production Index Japan: Electrical equipment',
                            'Production Index France: Electrical equipment',
                            'Production Index Germany: Electrical equipment']

shipment_index = ['Shipments Index Machinery & Electricals China',
                  'Shipments Index Machinery & Electricals France',
                  'Shipments Index Machinery & Electricals Germany',
                  'Shipments Index Machinery & Electricals Italy',
                  'Shipments Index Machinery & Electricals Japan',
                  'Shipments Index Machinery & Electricals Switzerland',
                  'Shipments Index Machinery & Electricals United Kingdom',
                  'Shipments Index Machinery & Electricals Europe']

world_price = [col for col in market_db1.columns if 'World: Price' in col]

producer_prices = ['Producer Prices United States: Electrical equipment',
                   'Producer Prices United Kingdom: Electrical equipment',
                   'Producer Prices Italy: Electrical equipment',
                   'Producer Prices France: Electrical equipment',
                   'Producer Prices Germany: Electrical equipment',
                   'Producer Prices China: Electrical equipment']

def plot_indicators(df, indicator_list, title):
    filtered_data = df[df['Indicator'].isin(indicator_list)]
    fig = px.line(filtered_data, x="DATE", y="Value", color="Indicator", title=title,
                  labels={"Value": "Cost in EUR" if "Price" in title else "Index Value", "DATE": "Date", "Indicator": "Indicator"})
    fig.show()

In [ ]:
market_melted = market_db1.melt(id_vars=["DATE"], var_name="Indicator", value_name="Value")
plot_indicators(market_melted, production_index_machinery_eletricals, "Production Index Trends Over Time")

The group observed that during the 2008–2009 financial crisis, most major economies saw a drop in their production indices relative to 2010 (baseline = 100). China, however, maintained higher production levels thanks to strong domestic demand and substantial government stimulus. Because the indices compare each month to its own 2010 baseline, China’s continued growth after 2010 results in a less pronounced dip compared to the sharper declines seen in other countries’ production levels. In other words, while most economies saw a steep production decline relative to 2010, China’s production levels, even during the crisis, stayed closer to or above its 2010 benchmark.

In [ ]:
g = sns.FacetGrid(market_melted[market_melted['Indicator'].isin(production_index_machinery_eletricals)], 
                  col="Indicator", col_wrap=3, sharey=False, height=4, aspect=2)
g.map(sns.lineplot, "DATE", "Value", color='#009999')

g.set_titles("{col_name}")
g.set_axis_labels("Date", "Value")
g.fig.tight_layout()
plt.show()

In [ ]:
map_db = {}

for col in production_index_machinery_eletricals:
    country = " ".join(col.split()[5:])
    map_db[country] = map_db.get(country, {})
    map_db[country]['Production Index Machinery & Electricals'] = market_db1[col].mean()

map_db['United States of America'] = map_db.pop('United States')
del map_db['Europe']

In [ ]:
data = map_db

df = pd.DataFrame.from_dict(data, orient='index').reset_index()
df.columns = ['Country', 'Production Index Machinery & Electricals']

world_geo = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"
).json()

m = folium.Map(location=[20, 0], zoom_start=2, tiles="cartodb positron")

colormap = linear.Reds_09.scale(
    df['Production Index Machinery & Electricals'].min(),
    df['Production Index Machinery & Electricals'].max()
)

choropleth = folium.Choropleth(
    geo_data=world_geo,
    name="choropleth",
    data=df,
    columns=["Country", 'Production Index Machinery & Electricals'],
    key_on="feature.properties.name",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    highlight=True,
    line_color='black',
    legend_name="Production Index Machine",
).add_to(m)

df['color'] = df['Production Index Machinery & Electricals'].apply(colormap)

style_function = lambda x: {
    'fillColor': '#f0f0f0',
    'color': '#d0d0d0',
    'fillOpacity': 0.2,
    'weight': 0.5
}
folium.GeoJson(
    world_geo,
    name="World Countries",
    style_function=style_function,
    control=False
).add_to(m)

choropleth.add_to(m)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], labels=False)
)

folium.LayerControl().add_to(m)

m

In [ ]:
plot_indicators(market_melted, production_index_machinery, "Production Index Machinery Trends Over Time")

In [ ]:
g = sns.FacetGrid(market_melted[market_melted['Indicator'].isin(production_index_machinery)], 
                  col="Indicator", col_wrap=3, sharey=False, height=4, aspect=2)
g.map(sns.lineplot, "DATE", "Value", color='#009999')
g.set_titles("{col_name}")
g.set_axis_labels("Date", "Value")
g.fig.tight_layout()
plt.show()

In [ ]:
map_db = {}
for col in production_index_machinery:
    country = ' '.join(col.split()[2:4])
    map_db[country] = map_db.get(country, {})
    map_db[country]['Machinery and equipment'] = market_db1[col].mean()

map_db['United States of America'] = map_db.pop('United States:')
map_db['United Kingdom'] = map_db.pop('United Kingdom:')
map_db['Switzerland'] = map_db.pop('Switzerland: Machinery')
map_db['Italy'] = map_db.pop('Italy: Machinery')
map_db['Japan'] = map_db.pop('Japan: Machinery')
map_db['France'] = map_db.pop('France: Machinery')
map_db[ 'Germany'] = map_db.pop( 'Germany: Machinery')
del map_db['World: Machinery']

map_db

In [ ]:
data = map_db

df = pd.DataFrame.from_dict(data, orient='index').reset_index()
df.columns = ['Country', 'Machinery and equipment']

world_geo = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"
).json()

m = folium.Map(location=[20, 0], zoom_start=2, tiles="cartodb positron")

colormap = linear.Reds_09.scale(
    df['Machinery and equipment'].min(),
    df['Machinery and equipment'].max()
)

choropleth = folium.Choropleth(
    geo_data=world_geo,
    name="choropleth",
    data=df,
    columns=["Country", 'Machinery and equipment'],
    key_on="feature.properties.name",
    fill_color='YlOrRd', 
    fill_opacity=0.7,
    line_opacity=0.2,
    highlight=True,
    line_color='black',
    legend_name="Production Index Machine",
).add_to(m)

df['color'] = df['Machinery and equipment'].apply(colormap)

style_function = lambda x: {
    'fillColor': '#f0f0f0', 
    'color': '#d0d0d0',   
    'fillOpacity': 0.2,
    'weight': 0.5
}


folium.GeoJson(
    world_geo,
    name="World Countries",
    style_function=style_function,
    control=False 
).add_to(m)

choropleth.add_to(m)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], labels=False)
)

folium.LayerControl().add_to(m)

m

In [ ]:
plot_indicators(market_melted, production_index_eletrical, "Production Index Electrical Equipment Trends Over Time")

In [ ]:
g = sns.FacetGrid(market_melted[market_melted['Indicator'].isin(production_index_eletrical)], 
                  col="Indicator", col_wrap=3, sharey=False, height=4, aspect=2)
g.map(sns.lineplot, "DATE", "Value", color='#009999')

g.set_titles("{col_name}")
g.set_axis_labels("Date", "Value")
g.fig.tight_layout()

plt.show()

In [ ]:
map_db = {}

for col in production_index_machinery:
    country = ' '.join(col.split()[2:4])  
    map_db[country] = map_db.get(country, {})
    map_db[country]['Electrical equipment'] = market_db1[col].mean()

map_db['United States of America'] = map_db.pop('United States:')
map_db['United Kingdom'] = map_db.pop('United Kingdom:')
map_db['Switzerland'] = map_db.pop('Switzerland: Machinery')
map_db['Italy'] = map_db.pop('Italy: Machinery')
map_db['Japan'] = map_db.pop('Japan: Machinery')
map_db['France'] = map_db.pop('France: Machinery')
map_db[ 'Germany'] = map_db.pop( 'Germany: Machinery')
del map_db['World: Machinery']

In [ ]:

data = map_db

df = pd.DataFrame.from_dict(data, orient='index').reset_index()
df.columns = ['Country', 'Electrical equipment']

world_geo = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"
).json()

m = folium.Map(location=[20, 0], zoom_start=2, tiles="cartodb positron")

colormap = linear.Reds_09.scale(
    df['Electrical equipment'].min(),
    df['Electrical equipment'].max()
)

choropleth = folium.Choropleth(
    geo_data=world_geo,
    name="choropleth",
    data=df,
    columns=["Country", 'Electrical equipment'],
    key_on="feature.properties.name",
    fill_color='YlOrRd',  
    line_opacity=0.2,
    highlight=True,
    line_color='black',
    legend_name="Electrical equipment",
).add_to(m)

df['color'] = df['Electrical equipment'].apply(colormap)

style_function = lambda x: {
    'fillColor': '#f0f0f0',  
    'color': '#d0d0d0',    
    'fillOpacity': 0.2,
    'weight': 0.5
}


folium.GeoJson(
    world_geo,
    name="World Countries",
    style_function=style_function,
    control=False  
).add_to(m)

choropleth.add_to(m)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], labels=False)
)

folium.LayerControl().add_to(m)

m

In [ ]:
plot_indicators(market_melted, shipment_index, "Shipments Index Trends Over Time")

In [ ]:
g = sns.FacetGrid(market_melted[market_melted['Indicator'].isin(shipment_index)], 
                  col="Indicator", col_wrap=3, sharey=False, height=4, aspect=2)
g.map(sns.lineplot, "DATE", "Value", color='#009999')

g.set_titles("{col_name}")
g.set_axis_labels("Date", "Value")
g.fig.tight_layout()

plt.show()

Regarding the Shipments, Germany, Italy, Japan, Switzerland, the UK, and Europe as a whole showed a sharp decline in shipments during 2008–2009. This drop reflects weaker global demand and disruptions in trade. China’s shipments keep climbing. Nearly all plots show a dip around early 2020, coinciding with lockdowns, supply chain disruptions, and reduced industrial activity. By late 2020 and into 2021, shipments rebound as economies reopen, although the pace varies by region.

In [ ]:
map_db = {}

for col in shipment_index:
    country = ' '.join(col.split()[5:])  
    map_db[country] = map_db.get(country, {})
    map_db[country]['Electrical equipment'] = market_db1[col].mean()

del map_db['Europe']

In [ ]:

data = map_db

df = pd.DataFrame.from_dict(data, orient='index').reset_index()
df.columns = ['Country', 'Shipments Index Machinery & Electricals']


world_geo = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"
).json()

m = folium.Map(location=[20, 0], zoom_start=2, tiles="cartodb positron")

# Create a custom color scale that uses reds instead of blues
colormap = linear.Reds_09.scale(
    df['Shipments Index Machinery & Electricals'].min(),
    df['Shipments Index Machinery & Electricals'].max()
)

# Add choropleth layer
choropleth = folium.Choropleth(
    geo_data=world_geo,
    name="choropleth",
    data=df,
    columns=["Country", 'Shipments Index Machinery & Electricals'],
    key_on="feature.properties.name",
    fill_color='YlOrRd',  # Use a predefined color scheme
    fill_opacity=0.7,
    line_opacity=0.2,
    highlight=True,
    line_color='black',
    legend_name="Shipments Index Machinery & Electricals",
).add_to(m)

df['color'] = df['Shipments Index Machinery & Electricals'].apply(colormap)

style_function = lambda x: {
    'fillColor': '#f0f0f0',  
    'color': '#d0d0d0',      
    'fillOpacity': 0.2,
    'weight': 0.5
}

folium.GeoJson(
    world_geo,
    name="World Countries",
    style_function=style_function,
    control=False 
).add_to(m)

choropleth.add_to(m)


choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], labels=False)
)


folium.LayerControl().add_to(m)


m

In [ ]:
plot_indicators(market_melted, world_price, "World Price Trends Over Time")

In [ ]:
plot_indicators(market_melted, producer_prices, "Producer Prices Trends Over Time")

In [ ]:
map_db = {}

for col in producer_prices:
    country = ' '.join(col.split()[2:4])  
    map_db[country] = map_db.get(country, {})
    map_db[country]['Producer Prices'] = market_db1[col].mean()

map_db['United States of America'] = map_db.pop('United States:')
map_db['United Kingdom'] = map_db.pop('United Kingdom:')
map_db['Italy'] = map_db.pop('Italy: Electrical')
map_db['France'] = map_db.pop('France: Electrical')
map_db[ 'Germany'] = map_db.pop( 'Germany: Electrical')
map_db['China'] = map_db.pop('China: Electrical')

In [ ]:

data = map_db


df = pd.DataFrame.from_dict(data, orient='index').reset_index()
df.columns = ['Country', 'Producer Prices']


world_geo = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"
).json()


m = folium.Map(location=[20, 0], zoom_start=2, tiles="cartodb positron")

colormap = linear.Reds_09.scale(
    df['Producer Prices'].min(),
    df['Producer Prices'].max()
)


choropleth = folium.Choropleth(
    geo_data=world_geo,
    name="choropleth",
    data=df,
    columns=["Country", 'Producer Prices'],
    key_on="feature.properties.name",
    fill_color='YlOrRd',  
    fill_opacity=0.7,
    line_opacity=0.2,
    highlight=True,
    line_color='black',
    legend_name="Producer Prices",
).add_to(m)


df['color'] = df['Producer Prices'].apply(colormap)


style_function = lambda x: {
    'fillColor': '#f0f0f0',
    'color': '#d0d0d0',      
    'fillOpacity': 0.2,
    'weight': 0.5
}


folium.GeoJson(
    world_geo,
    name="World Countries",
    style_function=style_function,
    control=False  
).add_to(m)

choropleth.add_to(m)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], labels=False)
)

folium.LayerControl().add_to(m)

m

From the plots about the commodity prices and producer prices the group observed some trends:
- 2008-2009 financial crisis: Around 2008, the price of the commodities (e.g. metals, energy) reached a peak and then fell as the global crisis unfolded. During this period, the industrial activity worldwide slowed down, reducing the demand for raw materials and leading to a drop in prices. Although commodity prices fell, producer prices sometimes show a lag or a different pace of change. In some regions or sectors, producer prices may have initially increased or remained elevated before eventually reflecting the downturn in input costs.

- Post-Crisis Recovery (2010-2011): After 2010/2011, both commodity and producer prices started to increase as the global economic activity started to improve. Emerging markets, in particular, fueled demand for raw materials, contributing to the recovery.

- 2014-2017: While producer prices continued an upward trend, reaching a peak around January 2017, some commodities experienced oversupply or softer demand, causing their prices to plateau or decrease.

- Covid-19 Impact (2020): Early 2020, lockdowns and reduced industrial output caused a big drop in many commodity and producer prices, reflecting the sudden contraction in economic activity. As economies reopened in 2021, pent-up demand and supply chain disruptions led to price increases. Commodities rebounded, and producer prices also surged in response to higher input costs and renewed consumer demand.

- 2021-2022 price surge: The price increases from 2021 to 2022 were the largest rise in the past 20 years, partly driven by disruptions in shipping, logistics, and labor markets, rising costs (e.g., energy, metals) feeding into producer prices and, ultimately, consumer prices (inflation). Also, as industries ramped up production, demand outpaced supply, further fueling price hikes.

#### Checking for characteristics and patterns

Just like the Sales Data, the Market Data was also decomposed to check to the Seasonality, Trends, Autocorrelation and Stationarity.

##### Seasonality and Trends

In [ ]:
market_db1

In [ ]:
market_copy = market_db1.copy()
market_db1 = market_db1.set_index("DATE")

In [ ]:
indicators = market_db1.columns.tolist()

In [ ]:
for i in production_index_machinery_eletricals:
    market_indicator = market_db1[i].to_frame()
    market_indicator = market_indicator.fillna(method='ffill').fillna(method='bfill')
    result = seasonal_decompose(market_indicator[i], model='additive', period=12)
    result.plot()
    plt.suptitle(f'Seasonal Decomposition for {i}', y=1.02)
    plt.show()

Regarding the trends in the production of machinery and eletricals, the product groups are distributed as follows:
- Uptrend: China
- Horizontal: France, Germany, Italy, Japan, Switzerland, United Kingdom, United Sates of America; Europe
- Downtredn: NA

In all indexes there is presence of seasonality.

In [ ]:
for i in production_index_machinery:
    market_indicator = market_db1[i].to_frame()
    market_indicator = market_indicator.fillna(method='ffill').fillna(method='bfill')
    result = seasonal_decompose(market_indicator[i], model='additive', period=12)
    result.plot()
    plt.suptitle(f'Seasonal Decomposition for {i}', y=1.02)
    plt.show()

Regarding the trends in the production of machinery and eletricals, the product groups are distributed as follows:
- Uptrend: NA
- Horizontal: United Sates of America; World; Switzerland; United Kingdom; Italy; Japan; France; Germany
- Downtredn: NA

In all indexes there is presence of seasonality.

In [ ]:
for i in shipment_index:
    market_indicator = market_db1[i].to_frame()
    market_indicator = market_indicator.fillna(method='ffill').fillna(method='bfill')
    result = seasonal_decompose(market_indicator[i], model='additive', period=12)
    result.plot()
    plt.suptitle(f'Seasonal Decomposition for {i}', y=1.02)
    plt.show()

Regarding the trends in the production of machinery and eletricals, the product groups are distributed as follows:
- Uptrend: China; Germany; Europe
- Horizontal: Switzerland; United Kingdom; Italy; Japan; France
- Downtredn: NA

In all indexes there is presence of seasonality.

In [ ]:
for i in world_price:
    market_indicator = market_db1[i].to_frame()
    market_indicator = market_indicator.fillna(method='ffill').fillna(method='bfill')
    result = seasonal_decompose(market_indicator[i], model='additive', period=12)
    result.plot()
    plt.suptitle(f'Seasonal Decomposition for {i}', y=1.02)
    plt.show()

Regarding the world prices, the indexes trending as follwos:
- Uptrend: Copper
- Horizontal: Base Metals; Price Energy; Metals & Minerals; Gas; Crude Oil
- Downtrend: NA

Again, all indexes show signs of seasonality.

##### Autocorrelation (ACF)

The group opted to comment this section to keep the notebook simpler and readable.

In [ ]:
# for i in indicators:
#     market_indicator = market_db1[[i]].fillna(method='ffill').fillna(method='bfill')
#     fig = plot_acf(market_indicator[i], title=f'Autocorrelation for Indicator {i}')
#     plt.show()

##### Stationarity (Dickey-Fuller test)

In [ ]:
market_db1_filtered = market_db1[market_db1.index >= '2018-10']

for i in production_index_eletrical:
    market_indicator = market_db1_filtered[[i]].fillna(method='ffill').fillna(method='bfill')
    adf, pval, usedlag, nobs, crit_vals, icbest = adfuller(market_indicator[i].values)
    
    print(f'Indicator: {i}')
    print('ADF test statistic:', adf)
    print('ADF p-value:', pval)
    print('Number of lags used:', usedlag)
    print('Number of observations:', nobs)
    print('Critical values:', crit_vals)
    print('Best information criterion:', icbest)
    print('\n')

In [ ]:
for i in shipment_index:
    market_indicator = market_db1_filtered[[i]].fillna(method='ffill').fillna(method='bfill')
    adf, pval, usedlag, nobs, crit_vals, icbest = adfuller(market_indicator[i].values)
    
    print(f'Indicator: {i}')
    print('ADF test statistic:', adf)
    print('ADF p-value:', pval)
    print('Number of lags used:', usedlag)
    print('Number of observations:', nobs)
    print('Critical values:', crit_vals)
    print('Best information criterion:', icbest)
    print('\n')

In [ ]:
for i in world_price:
    market_indicator = market_db1_filtered[[i]].fillna(method='ffill').fillna(method='bfill')
    adf, pval, usedlag, nobs, crit_vals, icbest = adfuller(market_indicator[i].values)
    
    print(f'Indicator: {i}')
    print('ADF test statistic:', adf)
    print('ADF p-value:', pval)
    print('Number of lags used:', usedlag)
    print('Number of observations:', nobs)
    print('Critical values:', crit_vals)
    print('Best information criterion:', icbest)
    print('\n')

In [ ]:
for i in producer_prices:
    market_indicator = market_db1_filtered[[i]].fillna(method='ffill').fillna(method='bfill')
    adf, pval, usedlag, nobs, crit_vals, icbest = adfuller(market_indicator[i].values)
    
    print(f'Indicator: {i}')
    print('ADF test statistic:', adf)
    print('ADF p-value:', pval)
    print('Number of lags used:', usedlag)
    print('Number of observations:', nobs)
    print('Critical values:', crit_vals)
    print('Best information criterion:', icbest)
    print('\n')

Many of your macroeconomic indicators show non-stationarity (p-value > 0.05), they were transformed to achieve stationarity before being used in the model.
- Production Index United States: Electrical equipment
- Production Index World: Electrical equipment
- Production Index Switzerland: Electrical equipment
- Production Index Italy: Electrical equipment
- Production Index Japan: Electrical equipment
- Production Index Germany: Electrical equipment
- Shipments Index Machinery & Electricals France
- Shipments Index Machinery & Electricals Germany
- Shipments Index Machinery & Electricals Japan
- Shipments Index Machinery & Electricals Switzerland
- Shipments Index Machinery & Electricals Europe
- World: Price of Base Metals
- World: Price of Energy
- World: Price of Metals & Minerals
- World: Price of Natural gas index
- World: Price of Crude oil, average
- World: Price of Copper
- Producer Prices United States: Electrical equipment
- Producer Prices United Kingdom: Electrical equipment
- Producer Prices Italy: Electrical equipment
- Producer Prices France: Electrical equipment
- Producer Prices Germany: Electrical equipment
- Producer Prices China: Electrical equipment

### Comparing Sales Data with Market Data

## 4. Data Cleaning

In [ ]:
sales_db = pd.read_csv("datasets\Case2_Sales data.csv", sep=";")
market_db = pd.read_excel("datasets\Case2_Market data.xlsx")

First, the group corrected the datatypes and converted daily to monthly.

In [ ]:
sales_db['Sales_EUR'] = sales_db['Sales_EUR'].str.replace(',', '.').astype(float)
sales_db.columns = sales_db.columns.str.strip()
sales_db['DATE'] = pd.to_datetime(sales_db['DATE'], dayfirst=True)
sales_db['DATE'] = sales_db['DATE'].dt.to_period('M')
sales_db = sales_db.groupby(['DATE', 'Mapped_GCK'])['Sales_EUR'].sum().reset_index()
sales_db

Regarding the market data, the group improved the structure of it.

In [ ]:
new_col_names = ['DATE',
                'Production Index Machinery & Electricals China', 'Shipments Index Machinery & Electricals China',
                'Production Index Machinery & Electricals France', 'Shipments Index Machinery & Electricals France',
                'Production Index Machinery & Electricals Germany', 'Shipments Index Machinery & Electricals Germany',
                'Production Index Machinery & Electricals Italy', 'Shipments Index Machinery & Electricals Italy',
                'Production Index Machinery & Electricals Japan', 'Shipments Index Machinery & Electricals Japan',
                'Production Index Machinery & Electricals Switzerland', 'Shipments Index Machinery & Electricals Switzerland',
                'Production Index Machinery & Electricals United Kingdom', 'Shipments Index Machinery & Electricals United Kingdom',
                'Production Index Machinery & Electricals United States', 'Shipments Index Machinery & Electricals United States',
                'Production Index Machinery & Electricals Europe', 'Shipments Index Machinery & Electricals Europe',
                'World: Price of Base Metals', 'World: Price of Energy', 'World: Price of Metals & Minerals', 'World: Price of Natural gas index', 'World: Price of Crude oil, average', 'World: Price of Copper',
                'United States: EUR in LCU',
                'Producer Prices United States: Electrical equipment',
                'Producer Prices United Kingdom: Electrical equipment',
                'Producer Prices Italy: Electrical equipment',
                'Producer Prices France: Electrical equipment',
                'Producer Prices Germany: Electrical equipment',
                'Producer Prices China: Electrical equipment',
                'Production Index United States: Machinery and equipment n.e.c.',
                'Prodcution Index World: Machinery and equipment n.e.c.',
                'Production Index Switzerland: Machinery and equipment n.e.c.',
                'Production Index United Kingdom: Machinery and equipment n.e.c.',
                'Production Index Italy: Machinery and equipment n.e.c.', 
                'Production Index  Japan: Machinery and equipment n.e.c.',
                'Production Index France: Machinery and equipment n.e.c.',
                'Production Index Germany: Machinery and equipment n.e.c.',
                'Production Index United States: Electrical equipment',
                'Production Index World: Electrical equipment',
                'Production Index Switzerland: Electrical equipment',
                'Production Index United Kingdom: Electrical equipment',
                'Production Index Italy: Electrical equipment',
                'Production Index Japan: Electrical equipment',
                'Production Index France: Electrical equipment','Production Index Germany: Electrical equipment']

market_db.columns = new_col_names
market_db.drop(0, inplace=True)
market_db.drop(1, inplace=True)
market_db.reset_index(drop=True, inplace=True)

Next, the group aligned the market dataset with the Sales dataset to make sure the indicators matched with the date range (2018-2022)

In [ ]:
market_db['DATE'] = pd.to_datetime(market_db['DATE'].str.strip().str.replace('m', '-'), format='%Y-%m')
market_db['DATE'] = market_db['DATE'].dt.to_period('M')
market_db = market_db[market_db['DATE'] >= '2018-10']
market_db

Since the sales were in EUR but some macroeconomic indicators like commodity prices are quoted in USD, the group decided to convert them into the same currency by dividing it by the exchange rate.

In [ ]:
price_columns = [
    'World: Price of Base Metals', 
    'World: Price of Energy', 
    'World: Price of Metals & Minerals',
    'World: Price of Natural gas index', 
    'World: Price of Crude oil, average', 
    'World: Price of Copper',
    'Producer Prices United States: Electrical equipment', 
    'Producer Prices United Kingdom: Electrical equipment',
    'Producer Prices Italy: Electrical equipment', 
    'Producer Prices France: Electrical equipment',
    'Producer Prices Germany: Electrical equipment', 
    'Producer Prices China: Electrical equipment'
]

for col in price_columns:
    market_db[col] = market_db[col] / market_db['United States: EUR in LCU']

market_db.head()

In [ ]:
nulls = market_db.isnull().sum()
nulls

Regarding the 2 columns with 18 missing rows each.

In [ ]:
nulls_list = nulls[nulls > 1].index.tolist()
nulls_list
for col in nulls_list:
    missing_dates = market_db[market_db[col].isnull()]['DATE'].dt.strftime('%Y-%m').unique()
    print(f"Missing dates for {col}: {missing_dates}")

The missing rows seems to be from "connected dates". The missing values started on October 2020 and end on April 2022, this is - about - half of the data and as such these features are best removed than treated.

The group also decided to remove the column Shipments China because it was exactly the same as the Production

In [ ]:
market_db = market_db.drop(columns=nulls_list + ['Shipments Index Machinery & Electricals China'])

Before handling the rest of the columns with missing values and the outliers, the group decided to split the dataset in train (80%) and validation (20%) preserving the chronological order of the dates.

In [ ]:
train_size = int(len(market_db) * 0.8)
market_train = market_db[:train_size]
market_validation = market_db[train_size:]

In [ ]:
sales_val = sales_db[sales_db['DATE'] >= '2021-08']
sales_train = sales_db[sales_db['DATE'] < '2021-08']

In [ ]:
nulls_market_val = market_validation.isnull().sum()
nulls_market_val

Lets treat the columns with one missing row by replacing it with the average for that index in that year.

In [ ]:
nulls_list = nulls_market_val[nulls_market_val > 0].index.tolist()

def filler(df, cols):
    for col in cols:
        if df[col].isna().sum() == 1:
            year = df[df[col].isnull()]['DATE'].dt.year.values[0]
            avg = df[df['DATE'].dt.year == year][col].mean()
            df[col].fillna(avg, inplace=True)
    return df

market_validation = filler(market_validation, nulls_list)

In [ ]:
for df in [market_train, market_validation]:
    obj_cols = df.select_dtypes(include='object').columns
    df[obj_cols] = df[obj_cols].astype(float)

In [ ]:
market_train.info()

In [ ]:
market_validation.info()

### Merging Data

In [ ]:
trans_train = sales_train.copy()
trans_val = sales_val.copy()

In [ ]:
train_df = sales_train.merge(market_train, on = "DATE", how = "left")
val_df = sales_val.merge(market_validation, on="DATE", how="left")

In [ ]:
train_df

In [ ]:
val_df

### Outlier Removal

In [ ]:
product_ids_to_check = ['#1', '#3', '#4', '#5', '#8', '#9', '#13', '#14', '#16' '#20', '#36']

filtered_train = train_df.copy()
filtered_trans = trans_train.copy()

for product_id in product_ids_to_check:
    
    product_data = filtered_train[filtered_train['Mapped_GCK'] == product_id]
    product_data1 = filtered_trans[filtered_trans['Mapped_GCK'] == product_id]
    
    Q1 = product_data['Sales_EUR'].quantile(0.25)
    Q3 = product_data['Sales_EUR'].quantile(0.75)
    IQR = Q3 - Q1

    Q11 = product_data1['Sales_EUR'].quantile(0.25)
    Q31 = product_data1['Sales_EUR'].quantile(0.75)
    IQR1 = Q3 - Q1
    
    lower_bound = Q1 - 1.6 * IQR
    upper_bound = Q3 + 1.6 * IQR
    
    lower_bound = Q11 - 1.6 * IQR1
    upper_bound = Q31 + 1.6 * IQR1

    for idx, row in product_data.iterrows():
        if row['Sales_EUR'] < lower_bound:
            filtered_train.at[idx, 'Sales_EUR'] = lower_bound  
            filtered_trans.at[idx, 'Sales_EUR'] = lower_bound
        elif row['Sales_EUR'] > upper_bound:
            filtered_train.at[idx, 'Sales_EUR'] = upper_bound  
            filtered_trans.at[idx, 'Sales_EUR'] = lower_bound

train_df = filtered_train
trans_train = filtered_trans

### Organizing the data

In [ ]:
mapped_gck_list = val_df['Mapped_GCK'].unique().tolist()

In [ ]:
train_product_dfs = {}  
val_product_dfs = {}
train_for_export = {}
val_for_export = {}

for i in mapped_gck_list:
    train_product_dfs[i] = train_df[train_df["Mapped_GCK"] == i]
    val_product_dfs[i] = val_df[val_df["Mapped_GCK"] == i]
    train_for_export[i] = train_df[train_df["Mapped_GCK"] == i]
    val_for_export[i] = val_df[val_df["Mapped_GCK"] == i]

### Differencing

To make the non-stationary series stationary, the group applied differencing, which removes trends and leaves only seasonal variations. Which helps when using models that assume stationarity. This process required the data to be merged and separated again, as it leads to the loss of one row and applying it in hte validation set would mean the loss of a step in the middle of the time series.

In [ ]:
train_val = pd.concat([train_df, val_df], axis=0)

In [ ]:
train_val_dfs = {}

for i in mapped_gck_list:
    train_val_dfs[i] = train_val[train_val["Mapped_GCK"] == i]

In [ ]:
def adf_test(series, alpha=0.05):
    result = ts.adfuller(series.dropna(), autolag='AIC')
    p_value = result[1]
    return p_value < alpha, p_value

transformations = {}

for product in train_val_dfs.keys():
    print(f"\nProcessing Product: {product}")
    
    df_train = train_val_dfs[product].copy()
    
    transformations[product] = {}
    
    exog_cols = [col for col in df_train.columns if col not in ['DATE', 'Mapped_GCK', 'Sales_EUR']]
    
    for col in exog_cols:
        is_stat, p_val = adf_test(df_train[col])
        if not is_stat:
            df_train[col] = df_train[col].diff()
            transformations[product][col] = 'first_difference'
            print(f"Column '{col}' for {product} non-stationary (p={p_val:.4f}). Applied differencing.")
        else:
            transformations[product][col] = 'none'
            print(f"Column '{col}' for {product} is stationary (p={p_val:.4f}).")
    
    df_train.dropna(inplace=True)
    train_val_dfs[product] = df_train

In [ ]:
for prod in train_val_dfs.keys():
    train_product_dfs[prod] = train_val_dfs[prod][train_val_dfs[prod]['DATE'] < '2021-08']
    val_product_dfs[prod] = train_val_dfs[prod][train_val_dfs[prod]['DATE'] >= '2021-08']

for prod in train_val_dfs.keys():
    train_for_export[prod] = train_val_dfs[prod][train_val_dfs[prod]['DATE'] < '2022-01']
    val_for_export[prod] = train_val_dfs[prod][train_val_dfs[prod]['DATE'] >= '2022-01']

## 5. Feature Selection

For the feature selection phase, the group decided to use the correlation matrix to identify the variables with a correlation higher than 0.90, the VIF, the Granger causality test to identify features that influence the target and feature importance with Random Forest.

In [ ]:
for key in train_product_dfs:
    train_product_dfs[key]['DATE'] = pd.to_datetime(train_product_dfs[key]['DATE'].astype(str))
    train_product_dfs[key].set_index('DATE', inplace=True)

    val_product_dfs[key]['DATE'] = pd.to_datetime(val_product_dfs[key]['DATE'].astype(str))
    val_product_dfs[key].set_index('DATE', inplace=True)

### Correlations

The group checked the subset of features that were highly correlated with each other and decided which ones to keep manually.

In [ ]:
for gck in train_product_dfs.keys():
    print(f"Processing Product: {gck}")
    
    train_gck = train_product_dfs[gck].drop(columns=['Mapped_GCK'])
    X_train = train_gck.drop(columns=['Sales_EUR'])

    corr_matrix = X_train.corr().abs()

    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    high_corr_pairs = []
    for column in upper.columns:
        high_corr = upper[column][upper[column] > 0.9]
        for idx in high_corr.index:
            high_corr_pairs.append((column, idx, high_corr[idx]))

    if high_corr_pairs:
        print("Highly correlated feature pairs (correlation > 0.9):")
        for feature1, feature2, corr_value in high_corr_pairs:
            print(f"{feature1} ↔ {feature2} (Correlation: {corr_value:.2f})")
    else:
        print("No highly correlated feature pairs found.")

    print("-" * 50) 

Features to drop:
- Production Index Machinery & Electricals Europe
- Shipments Index Machinery & Electricals Europe
- Shipments Index Machinery & Electricals Switzerland
- Shipments Index Machinery & Electricals Japan
- Shipments Index Machinery & Electricals Germany

- World: Price of Crude oil, average
- World: Price of Base Metals

- United States: EUR in LCU
- Producer Prices Italy: Electrical equipment
- Producer Prices France: Electrical equipment
- Producer Prices United States: Electrical equipment
- The group decided to retain only "Producer Prices Germany: Electrical equipment" because the other producer price indicators were highly correlated with each other and using a local producer price index ensures that your model reflects the environment in which the business unit operates.

- Production Index United States: Machinery and equipment n.e.c.
- Prodcution Index World: Machinery and equipment n.e.c.
- Production Index Switzerland: Machinery and equipment n.e.c.
- Production Index United Kingdom: Machinery and equipment n.e.c.
- Production Index Italy: Machinery and equipment n.e.c.
- Production Index  Japan: Machinery and equipment n.e.c.
- Production Index France: Machinery and equipment n.e.c.

- Production Index World: Electrical equipment
- Production Index Switzerland: Electrical equipment
- Production Index Italy: Electrical equipment
- Production Index Japan: Electrical equipment

In [ ]:
features_to_drop = [
    "Production Index Machinery & Electricals Europe",
    "Shipments Index Machinery & Electricals Europe",
    "Shipments Index Machinery & Electricals Switzerland",
    "Shipments Index Machinery & Electricals Japan",
    "Shipments Index Machinery & Electricals Germany",
    "World: Price of Crude oil, average",
    "World: Price of Base Metals",
    "United States: EUR in LCU",
    "Producer Prices Italy: Electrical equipment",
    "Producer Prices France: Electrical equipment",
    "Producer Prices United States: Electrical equipment",
    "Production Index United States: Machinery and equipment n.e.c.",
    "Prodcution Index World: Machinery and equipment n.e.c.",
    "Production Index Switzerland: Machinery and equipment n.e.c.",
    "Production Index United Kingdom: Machinery and equipment n.e.c.",
    "Production Index Italy: Machinery and equipment n.e.c.",
    "Production Index  Japan: Machinery and equipment n.e.c.",
    "Production Index France: Machinery and equipment n.e.c.",
    "Production Index World: Electrical equipment",
    "Production Index Switzerland: Electrical equipment",
    "Production Index Italy: Electrical equipment",
    "Production Index Japan: Electrical equipment"
]

for gck in train_product_dfs.keys():
    train_product_dfs[gck] = train_product_dfs[gck].drop(columns=features_to_drop)
    val_product_dfs[gck] = val_product_dfs[gck].drop(columns=features_to_drop)

### VIF
VIF feature selection identifies and removes highly collinear variables by calculating how much a predictor (a variable) is inflated by correlations with other predictors. Features with high VIF values (in this case >10) are removed to reduce multicollinearity.

In [ ]:
def high_vif(data, threshold=10):
    vif_data = pd.DataFrame()
    vif_data["feature"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    print("VIF values:\n", vif_data)
    return vif_data[vif_data["VIF"] < threshold]["feature"].tolist()

for gck in train_product_dfs.keys():
    print(f"Selecting features for Product: {gck}")
    train_gck = train_product_dfs[gck].drop(columns=['Mapped_GCK'])
    y_train = train_gck['Sales_EUR']
    X_train = train_gck.drop(columns=['Sales_EUR'])
    selected_features = high_vif(X_train)

### Granger Test
Next, the group used the granger causality test to determine wheter one time series can predict the future values of another time series. "It measures the extent to which the past values of one variable provide valuable information for forecasting the other variable’s future behavior" (Padav, 2024)<sup>1</sup>.

In [ ]:
def granger_test(data, target, max_lag=5, significance=0.05):
    selected_features = []
    for col in data.columns:
        test_result = grangercausalitytests(data[[target, col]], max_lag, verbose=False)
        p_values = [test_result[i+1][0]['ssr_ftest'][1] for i in range(max_lag)]
        if min(p_values) < significance:
            selected_features.append(col)
    return selected_features

for gck in train_product_dfs.keys():
    print(f"Selecting features for Product: {gck}")

    train_gck = train_product_dfs[gck].drop(columns=['Mapped_GCK'])
    y_train = train_gck['Sales_EUR']
    X_train = train_gck.drop(columns=['Sales_EUR'])
    selected_features = granger_test(train_gck, 'Sales_EUR')
    print("Features selected by Granger Causality Test:", selected_features, "\n")

### Feature Importance

To decide the final indicators for each product, the group used feature importance using Random Forest which identifies the most influential features. (Note: It was used a threshold for each product)

In [ ]:
def feature_importance(X, y):
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X, y)
    return rf.feature_importances_

for gck in train_product_dfs.keys():
    print(f"Selecting features for Product: {gck}")

    train_gck = train_product_dfs[gck].drop(columns=['Mapped_GCK'])
    y_train = train_gck['Sales_EUR']
    X_train = train_gck.drop(columns=['Sales_EUR'])

    rf_importance = feature_importance(X_train, y_train)
    rf_features = X_train.columns[np.argsort(rf_importance)[::-1]].tolist()

    importance_df = pd.DataFrame({
        "Feature": rf_features,
        "Random_Forest_Importance": rf_importance
    })

    importance_df = importance_df.sort_values(by = "Random_Forest_Importance", ascending= False)

    plt.figure(figsize=(7,6))
    plt.barh(importance_df["Feature"], importance_df["Random_Forest_Importance"], color="#009999", label="Random Forest")
    plt.xlabel("Feature Importance")
    plt.ylabel("Features")
    plt.title("Feature Importance Random Forest")
    plt.legend()
    plt.gca().invert_yaxis()
    plt.show()

### Exogenous Variables for Each GCK
The group used an excel file (available in the [github](https://github.com/mikefferreira/BusinessCases)) to the decide the features to keep and drop.

- **#1**:
  - Production Index Machinery & Electricals China  
  - Production Index Machinery & Electricals Switzerland  
  - World: Price of Energy  
  - Production Index United States: Electrical equipment  
  - Production Index France: Electrical equipment  

- **#11**:  
  - World: Price of Metals & Minerals  

- **#12**:  
  - Shipments Index Machinery & Electricals France  
  - Production Index Machinery & Electricals Italy  
  - Production Index United States: Electrical equipment  

- **#13**:  
  - None  

- **#14**:  
  - Production Index Machinery & Electricals Switzerland  
  - World: Price of Energy  

- **#20**:  
  - Production Index Machinery & Electricals China  
  - Production Index Machinery & Electricals Germany  
  - Production Index Machinery & Electricals Japan  
  - World: Price of Copper  
  - World: Price of Metals & Minerals  
  - World: Price of Natural gas index  
  - Production Index United Kingdom: Electrical equipment  

- **#3**:  
  - Production Index Machinery & Electricals United States  
  - World: Price of Copper  

- **#36**:  
  - Production Index Machinery & Electricals United States  

- **#4**:  
  - Production Index Machinery & Electricals Germany  
  - Production Index Machinery & Electricals Japan  
  - World: Price of Natural gas index  
  - Production Index Germany: Machinery and equipment n.e.c.  

- **#5**:  
  - Production Index Machinery & Electricals Germany  
  - Production Index Machinery & Electricals Japan  
  - Producer Prices Germany: Electrical equipment  
  - Production Index United States: Electrical equipment  

- **#6**:  
  - Production Index Machinery & Electricals China  
  - World: Price of Copper  
  - World: Price of Energy  

- **#8**:  
  - Production Index Machinery & Electricals China  
  - Shipments Index Machinery & Electricals Italy  
  - Production Index Machinery & Electricals Switzerland  
  - World: Price of Copper  
  - Production Index Germany: Machinery and equipment n.e.c.  
  - Production Index United States: Electrical equipment  

- **#9**:  
  - World: Price of Copper  

- **#16**:  
  - None  

## 6. Modelation

### ARIMA (without exogenous variables)

"​An Autoregressive Integrated Moving Average (ARIMA) model is a statistical analysis tool that utilizes time series data to either better understand the dataset or to predict future trends."(Adam Hayes, 2024)<sup>2</sup> 

This model is quite simplistic and it served as a benchmark before adding macroeconomic indicators and moving to more complex - and hopefully better performing - models.

In [ ]:
results = {}
for gck in train_product_dfs.keys():
    print(f"Training ARIMA for Product: {gck}")

    train_gck = train_product_dfs[gck]
    val_gck = val_product_dfs[gck]
    
    train_gck = train_gck.drop(columns=['Mapped_GCK'])
    val_gck = val_gck.drop(columns=['Mapped_GCK'])
    
    y_train = train_gck['Sales_EUR']
    y_val = val_gck['Sales_EUR']

    model = ARIMA(y_train, order=(1, 1, 1))
    model_fit = model.fit()

    y_pred = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_val) - 1)

    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    results[gck] = {'rmse': rmse, 'y_val': y_val, 'y_pred': y_pred}
    
    print(f"Product: {gck}, RMSE: {rmse}")
    
    plt.figure(figsize=(12, 6))
    plt.plot(y_val.index, y_val, label='Actual Sales')
    plt.plot(y_val.index, y_pred, label='Predicted Sales', linestyle='dashed', color='#2D373C')
    plt.xlabel('Date')
    plt.ylabel('Sales (EUR)')
    plt.title(f'ARIMA Model - Actual vs Predicted Sales for {gck}')
    plt.legend()
    plt.show()

for gck, result in results.items():
    print(f"Product: {gck}, RMSE: {result['rmse']}")

This model produced poor results; however, as previously stated, it served as only has a benchmark.

### SARIMAX (using all exogenous variables)

"The Seasonal Autoregressive Integrated Moving Average with Exogenous Regressors (SARIMAX) model is a powerful time series forecasting technique that extends the traditional ARIMA model to account for seasonality and external factors. It's a versatile model that can accommodate both autoregressive (AR) and moving average (MA) components, integrate differencing to make the data stationary, and incorporate external variables or regressors. SARIMAX is particularly valuable when dealing with time-dependent data that exhibits recurring patterns over specific time intervals."(GeekforGeeks, 2023)<sup>3</sup>

Before trying a SARIMAX model with the feature list that was created, the group decided to create yet another baseline prediction where all possible features were used.

In [ ]:
results = {}
for gck in train_product_dfs.keys():
    print(f"Training SARIMAX for Product: {gck}")

    train_gck = train_product_dfs[gck]
    val_gck = val_product_dfs[gck]

    train_gck = train_gck.drop(columns=['Mapped_GCK'])
    val_gck = val_gck.drop(columns=['Mapped_GCK'])

    y_train = train_gck['Sales_EUR']
    X_train = train_gck.drop(columns=['Sales_EUR'])

    y_val = val_gck['Sales_EUR']
    X_val = val_gck.drop(columns=['Sales_EUR'])

    model = SARIMAX(y_train, exog=X_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)

    y_pred = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_val) - 1, exog=X_val)

    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    results[gck] = {'rmse': rmse, 'y_val': y_val, 'y_pred': y_pred}

    print(f"Product: {gck}, RMSE: {rmse}")

    plt.figure(figsize=(12, 6))
    plt.plot(y_val.index, y_val, label='Actual Sales')
    plt.plot(y_val.index, y_pred, label='Predicted Sales', linestyle='dashed', color='#2D373C')
    plt.xlabel('Date')
    plt.ylabel('Sales (EUR)')
    plt.title(f'SARIMAX Model - Actual vs Predicted Sales for {gck}')
    plt.legend()
    plt.show()

for gck, result in results.items():
    print(f'Product: {gck}, RMSE: {result["rmse"]}')

After incorporating all exogenous variables, the group observed that the overall results were not optimal. However, for certain products, the RMSE values were within a reasonable range, indicating some potential benefits of including the variables.

### SARIMAX

The group then proceeded with the SARIMAX model using the previously selected features. However, since the feature selection process indicated that no additional variables were needed for products #16 and #13, these were modeled without exogenous variables.

In [ ]:
results = {}

gck_exog_vars = {
    '#1': ['Production Index Machinery & Electricals China', 'Production Index Machinery & Electricals Switzerland', 'World: Price of Energy', 'Production Index United States: Electrical equipment', 'Production Index France: Electrical equipment'],
    '#11': ['World: Price of Metals & Minerals'],
    '#12': ['Shipments Index Machinery & Electricals France', 'Production Index Machinery & Electricals Italy', 'Production Index United States: Electrical equipment'],
    '#13': None,
    '#14': ['Production Index Machinery & Electricals Switzerland', 'World: Price of Energy'],
    '#20': ['Production Index Machinery & Electricals China', 'Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'World: Price of Copper', 'World: Price of Metals & Minerals', 'World: Price of Natural gas index', 'Production Index United Kingdom: Electrical equipment'],
    '#3': ['Production Index Machinery & Electricals United States', 'World: Price of Copper'],
    '#36': ['Production Index Machinery & Electricals United States'],
    '#4': ['Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'World: Price of Natural gas index', 'Production Index Germany: Machinery and equipment n.e.c.'],
    '#5': ['Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'Producer Prices Germany: Electrical equipment', 'Production Index United States: Electrical equipment'],
    '#6': ['Production Index Machinery & Electricals China', 'World: Price of Copper', 'World: Price of Energy'],
    '#8': ['Production Index Machinery & Electricals China', 'Shipments Index Machinery & Electricals Italy', 'Production Index Machinery & Electricals Switzerland', 'World: Price of Copper', 'Production Index Germany: Machinery and equipment n.e.c.', 'Production Index United States: Electrical equipment'],
    '#9': ['World: Price of Copper'],
    '#16' : None,
}

for gck in train_product_dfs.keys():
    print(f"Training ARIMAX for Product: {gck}")

    train_gck = train_product_dfs[gck]
    val_gck = val_product_dfs[gck]

    train_gck = train_gck.drop(columns=['Mapped_GCK'])
    val_gck = val_gck.drop(columns=['Mapped_GCK'])

    y_train = train_gck['Sales_EUR']
    y_val = val_gck['Sales_EUR']

    if gck_exog_vars[gck] is not None:
        X_train = train_gck[gck_exog_vars[gck]]
        X_val = val_gck[gck_exog_vars[gck]]

        model = SARIMAX(y_train, exog=X_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        model_fit = model.fit(disp=False)

        y_pred = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_val) - 1, exog=X_val)
    else:
        model = ARIMA(y_train, order=(1, 1, 1))
        model_fit = model.fit()
        y_pred = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_val) - 1)

    y_pred = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_val) - 1, exog=X_val)

    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    results[gck] = {'rmse': rmse, 'y_val': y_val, 'y_pred': y_pred}

    print(f"Product: {gck}, RMSE: {rmse}")

    plt.figure(figsize=(12, 6))
    plt.plot(y_val.index, y_val, label='Actual Sales')
    plt.plot(y_val.index, y_pred, label='Predicted Sales', linestyle='dashed', color='#2D373C')
    plt.xlabel('Date')
    plt.ylabel('Sales (EUR)')
    plt.title(f'ARIMAX Model - Actual vs Predicted Sales for {gck}')
    plt.legend()
    plt.show()

for gck, result in results.items():
    print(f'Product: {gck}, RMSE: {result["rmse"]}')

sarimax_rmse = {gck: result['rmse'] for gck, result in results.items() if 'rmse' in result}

Next, the group implemented SARIMAX with selected exogenous features for each product and an ARIMAX for the products #13 and #16:

Best Performing Models (Lowest RMSE):
- Product #20 (RMSE: 3671)
- Product #9 (RMSE: 8973)
- Product #13 (RMSE: 10044)
- Product #14 (RMSE: 14513)
- Product: #36 (RMSE: 16894)
- Product #16 (RMSE: 88102)

These products exhibited strong predictive performance, suggesting that the selected exogenous variables effectively contributed to capturing demand patterns.

Moderate Performance:
- Products such as #11 (RMSE: 644670), #12 (RMSE: 317041), #8 (RMSE: 622064), #6 (RMSE: 380950), and #4 (RMSE: 419306) had RMSE values in the mid-range.

Underperforming Models (Highest RMSE):
- Product #1 (RMSE: 6294103)
- Product #5 (RMSE: 4911513)
- Product #3 (RMSE: 2225187)

The high RMSE values for these products suggest that the selected exogenous variables might not be sufficient to explain the variation in sales.

### XGBoost

"XGBoost is short for Extreme Gradient Boosting and is an efficient implementation of the stochastic gradient boosting machine learning algorithm. The stochastic gradient boosting algorithm, also called gradient boosting machines or tree boosting, is a powerful machine learning technique that performs well or even best on a wide range of challenging machine learning problems." (Jason Brownlee, 2021)

In [ ]:
for product in train_product_dfs.keys():
    train_product_dfs[product] = train_product_dfs[product].reset_index()
    val_product_dfs[product] = val_product_dfs[product].reset_index()

In [ ]:
from darts import TimeSeries
from darts.models import XGBModel
from darts.metrics import rmse
from darts.models.forecasting.xgboost import XGBModel

forecasts = {}
rmse_scores = {}

for gck in train_product_dfs.keys():
    print(f"Training XGBoost for Product: {gck}")

    train_gck = train_product_dfs[gck].copy()
    val_gck = val_product_dfs[gck].copy()

    train_gck = train_gck.drop(columns=['Mapped_GCK'])
    val_gck = val_gck.drop(columns=['Mapped_GCK'])

    y_train = train_gck['Sales_EUR']
    y_val = val_gck['Sales_EUR']

    target_train = TimeSeries.from_dataframe(train_gck, time_col='DATE', value_cols=['Sales_EUR'])
    target_val = TimeSeries.from_dataframe(val_gck, time_col='DATE', value_cols=['Sales_EUR'])

    if gck in selected_features:
        X_train = train_gck[selected_features[gck]]
        X_val = val_gck[selected_features[gck]]

        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        exog_train = TimeSeries.from_dataframe(train_gck, time_col='DATE', value_cols=selected_features[gck])
        exog_val = TimeSeries.from_dataframe(val_gck, time_col='DATE', value_cols=selected_features[gck])
    else:
        exog_train, exog_val = None, None  


    model = XGBModel(lags=9, lags_exog=9 if exog_train else None)
    model.fit(target_train, past_covariates=exog_train)

    y_pred = model.predict(n=9, past_covariates=exog_val)

    forecasts[gck] = y_pred

    rmse_value = rmse(target_val, y_pred)
    rmse_scores[gck] = rmse_value

    plt.figure(figsize=(10, 5))
    target_val.plot(label="Actual Sales", linestyle="-", color = '#009999')
    y_pred.plot(label="Predicted Sales", linestyle="--", color= '#2D373C')
    plt.title(f"Product {gck}: Actual vs Predicted Sales (RMSE: {rmse_value:.2f})")
    plt.xlabel("Time")
    plt.ylabel("Sales_EUR")
    plt.legend()
    plt.grid(True)
    plt.show()

for gck, rmse_value in rmse_scores.items():
    print(f'Product: {gck}, RMSE: {rmse_value}')

xgb_rmse = {gck: rmse_value for gck, rmse_value in rmse_scores.items() if rmse_value is not None}

Next, the group implemented XGBoost with selected exogenous features for each product:

Best Performing Models (Lowest RMSE):
- Product #9 (RMSE: 7156)
- Product #20 (RMSE: 3975)
- Product #13 (RMSE: 15815)
- Product #14 (RMSE: 13729)
- Product #36 (RMSE: 16268)
- Product #16 (RMSE: 72988)

Moderate Performance:
- Products such as #12 (RMSE: 159161), #6 (RMSE: 319482), #4 (RMSE: 192749), #8 (RMSE: 705728), and #11 (RMSE: 1449084) had RMSE values in the mid-range.

Underperforming Models (Highest RMSE):
- Product #1 (RMSE: 2881129)
- Product #3 (RMSE: 2828943)
- Product #5 (RMSE: 5272665)

### Prophet

"Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well." (GitHub, n.d.)<sup>4</sup>

This model is open-source software released by Meta (formerly known as Facebook) and is widely known for its efficiency and proficiency.

In [ ]:
prophet_results = {}

for product in mapped_gck_list:
    print(f"Processing Product: {product}")
    
    df_train = train_product_dfs[product].reset_index()
    df_val   = val_product_dfs[product].reset_index()
    
    df_prophet = df_train[['DATE', 'Sales_EUR']].rename(columns={'DATE': 'ds', 'Sales_EUR': 'y'})
    
    model = Prophet(seasonality_mode='multiplicative', yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    model.fit(df_prophet)
    
    forecast_periods = len(df_val)
    future = model.make_future_dataframe(periods=forecast_periods, freq='MS')
    
    forecast = model.predict(future)

    y_pred = forecast['yhat'][-forecast_periods:]

    mse = mean_squared_error(df_val['Sales_EUR'], y_pred)
    rmse = np.sqrt(mse)

    prophet_results[product] = {'rmse': rmse, 'y_val': df_val['Sales_EUR'], 'y_pred': y_pred}

    print(f"Product: {product}, RMSE: {rmse}")

    plt.figure(figsize=(12, 6))
    plt.plot(df_val['DATE'], df_val['Sales_EUR'], label='Actual Sales', color = '#009999')
    plt.plot(df_val['DATE'], y_pred, label='Predicted Sales', linestyle='dashed', color='#2D373C')
    plt.xlabel('Date')
    plt.ylabel('Sales (EUR)')
    plt.title(f'Prophet Model - Actual vs Predicted Sales for {product}')
    plt.legend()
    plt.show()

for product, result in prophet_results.items():
    print(f"Product: {product}, RMSE: {result['rmse']}")

prophet_rmse = {gck: rmse_value for gck, rmse_value in prophet_results.items() if rmse_value is not None}

Next, the group implemented Prophet

Best Performing Models (Lowest RMSE):
- Product #20 (RMSE: 4371)
- Product #9 (RMSE: 12507)
- Product #13 (RMSE: 15307)
- Product #14 (RMSE: 14480)
- Product #36 (RMSE: 16034)


Moderate Performance:
- Products such as #12 (RMSE: 231233), #6 (RMSE: 279889), #16 (RMSE: 356636), #4 (RMSE: 179709), #8 (RMSE: 544612), and #11 (RMSE: 704133) had RMSE values in the mid-range.

Underperforming Models (Highest RMSE):
- Product #1 (RMSE: 4243981)
- Product #5 (RMSE: 4247702)
- Product #3 (RMSE: 2477892)

### TIMEGPT

"TimeGPT is a production-ready generative pretrained transformer for time series. It’s capable of accurately predicting various domains such as retail, electricity, finance, and IoT with just a few lines of code." (NIXTLA, 2024)<sup>5</sup>

In [ ]:
nixtla_client = NixtlaClient(
    api_key = 'nixak-IJDy3xU0F51O6p8oD2ZxTm1f0qOuwPofNu8bXP1NpIpGj8O5uVN8WE3YecfIJh0DLMi2AMof6672XL4t')

In [ ]:
nixtla_client.validate_api_key()

In [ ]:
trans_train['DATE'] = pd.to_datetime(trans_train['DATE'].dt.to_timestamp())
trans_val['DATE'] = pd.to_datetime(trans_val['DATE'].dt.to_timestamp())

In [ ]:
timegpt_fcst_df = nixtla_client.forecast(trans_train, time_col='DATE', target_col='Sales_EUR', id_col='Mapped_GCK', h = 9, freq='MS', model='timegpt-1-long-horizon')
timegpt_fcst_df.head()

In [ ]:
unique_gcks = timegpt_fcst_df['Mapped_GCK'].unique()

for gck in unique_gcks:
    fig, ax = plt.subplots(figsize=(18, 4))
    sns.set_style('whitegrid')
    
    actual_data = trans_val[trans_val['Mapped_GCK'] == gck]
    forecasted_data = timegpt_fcst_df[timegpt_fcst_df['Mapped_GCK'] == gck]
    
    sns.lineplot(x='DATE', y='Sales_EUR', data=actual_data, color='#009999', label='Actual Sales')
    sns.lineplot(x='DATE', y='TimeGPT', data=forecasted_data, color='#2D373C', label='Forecasted Sales', linestyle='dashed')
    
    sns.despine(bottom=True, left=True)
    plt.title(f'Actual vs Forecasted Sales for GCK {gck}', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Sales (EUR)', fontsize=12)
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()

In [ ]:
timegpt_rmse = {}
for gck in unique_gcks:
    actual_sales = trans_val[trans_val['Mapped_GCK'] == gck]['Sales_EUR']
    forecasted_sales = timegpt_fcst_df[timegpt_fcst_df['Mapped_GCK'] == gck]['TimeGPT']
    timegpt_rmse[gck] = root_mean_squared_error(actual_sales, forecasted_sales)

timegpt_rmse = {gck: rmse_value for gck, rmse_value in timegpt_rmse.items() if rmse_value is not None}
for i in timegpt_rmse:
    print(i, timegpt_rmse[i])

Next, the group implemented TimeGPT:

Best Performing Models (Lowest RMSE):
- Product #20 (RMSE: 1939) – Best overall performance
- Product #9 (RMSE: 8737)
- Product #13 (RMSE: 16146)
- Product #36 (RMSE: 16070)

Moderate Performance:
- Products such as #12 (RMSE: 146846), #16 (RMSE: 91280), #6 (RMSE: 267882), #4 (RMSE: 69556), #8 (RMSE: 544870), and #11 (RMSE: 696550) had RMSE values in the mid-range.

Underperforming Models (Highest RMSE):
- Product #1 (RMSE: 3725921)
- Product #5 (RMSE: 4162117)
- Product #3 (RMSE: 2064101)

### Neural Forecast

"NeuralForecast offers a large collection of neural forecasting models focusing on their performance, usability, and robustness." (Github, n.d.)<sup>5</sup>

In [ ]:
gck_features = {
    '#1': ['Production Index Machinery & Electricals China', 'Production Index Machinery & Electricals Switzerland', 'World: Price of Energy', 'Production Index United States: Electrical equipment', 'Production Index France: Electrical equipment'],
    '#11': ['World: Price of Metals & Minerals'],
    '#12': ['Shipments Index Machinery & Electricals France', 'Production Index Machinery & Electricals Italy', 'Production Index United States: Electrical equipment'],
    '#13': None,
    '#14': ['Production Index Machinery & Electricals Switzerland', 'World: Price of Energy'],
    '#20': ['Production Index Machinery & Electricals China', 'Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'World: Price of Copper', 'World: Price of Metals & Minerals', 'World: Price of Natural gas index', 'Production Index United Kingdom: Electrical equipment'],
    '#3': ['Production Index Machinery & Electricals United States', 'World: Price of Copper'],
    '#36': ['Production Index Machinery & Electricals United States'],
    '#4': ['Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'World: Price of Natural gas index', 'Production Index Germany: Machinery and equipment n.e.c.'],
    '#5': ['Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'Producer Prices Germany: Electrical equipment', 'Production Index United States: Electrical equipment'],
    '#6': ['Production Index Machinery & Electricals China', 'World: Price of Copper', 'World: Price of Energy'],
    '#8': ['Production Index Machinery & Electricals China', 'Shipments Index Machinery & Electricals Italy', 'Production Index Machinery & Electricals Switzerland', 'World: Price of Copper', 'Production Index Germany: Machinery and equipment n.e.c.', 'Production Index United States: Electrical equipment'],
    '#9': ['World: Price of Copper'],
    '#16' : None
    }

gck_predictions = {}

for gck, features in gck_features.items():
    print(f"Processing GCK: {gck}")
    
    gck_train = train_product_dfs[gck].reset_index()
    gck_train = gck_train.rename(columns={'DATE': 'ds', 'Sales_EUR': 'y', 'Mapped_GCK': 'unique_id'})
    gck_train = gck_train[['ds', 'y', 'unique_id'] + (features if features else [])]
    
    models = [
        NBEATS(input_size=2 * 10, h=10, max_steps=100, enable_progress_bar=False),
        NHITS(input_size=2 * 10, h=10, max_steps=100, enable_progress_bar=False)
    ]
    nf = NeuralForecast(models=models, freq='MS')
    
    nf.fit(df=gck_train)
    
    Y_hat_df = nf.predict()
    gck_predictions[gck] = Y_hat_df

    plt.figure(figsize=(10, 6))
    sns.lineplot(data=val_product_dfs[gck].reset_index(), x='DATE', y='Sales_EUR', label='Actual Sales', color='#009999')
    sns.lineplot(data=Y_hat_df, x='ds', y='NBEATS', label='NBEATS', marker='x', color='#2D373C', linestyle='dashed')
    sns.lineplot(data=Y_hat_df, x='ds', y='NHITS', label='NHITS', marker='x', color='#E3700F',  linestyle='dashed')
    plt.xlabel('Date')
    plt.ylabel('Prediction')
    plt.title(f'Predicted Time Series for GCK {gck}')
    plt.legend()
    plt.show()

In [ ]:
nbeats_rmse = {}
nhits_rmse = {}

for gck in unique_gcks:
    actual_sales = val_product_dfs[gck]['Sales_EUR']
    nbeats_sales = gck_predictions[gck]['NBEATS'][:len(actual_sales)] 
    nhits_sales = gck_predictions[gck]['NHITS'][:len(actual_sales)] 
    
    nbeats_rmse[gck] = root_mean_squared_error(actual_sales, nbeats_sales)
    nhits_rmse[gck] = root_mean_squared_error(actual_sales, nhits_sales)

nbeats_rmse = {gck: rmse_value for gck, rmse_value in nbeats_rmse.items() if rmse_value is not None}
nhits_rmse = {gck: rmse_value for gck, rmse_value in nhits_rmse.items() if rmse_value is not None}

for i in nbeats_rmse:
    print(f"NBEATS RMSE for {i}: {nbeats_rmse[i]}")
for i in nhits_rmse:
    print(f"NHITS RMSE for {i}: {nhits_rmse[i]}")

Next, the group implemented Neural Forecast (NBEATS & NHITS) with selected exogenous features for each product.

Best Performing Models (Lowest RMSE):
- Product #20 – NBEATS (RMSE: 2551) | NHITS (RMSE: 2644)
- Product #9 – NBEATS (RMSE: 6755) | NHITS (RMSE: 7629)
- Product #13 – NBEATS (RMSE: 16583) | NHITS (RMSE: 17185)
- Product #36 – NBEATS (RMSE: 16152) | NHITS (RMSE: 17350)

Moderate Performance:
- Products such as #12 (RMSE: NBEATS 224116 | NHITS 195330), #6 (RMSE: NBEATS 242462 | NHITS 239003), #4 (RMSE: NBEATS 223699 | NHITS 215436), and #8 (RMSE: NBEATS 654237 | NHITS 659060) had mid-range RMSE values.

Underperforming Models (Highest RMSE):
- Product #1 – NBEATS (RMSE: 4068513) | NHITS (RMSE: 3596081)
- Product #5 – NBEATS (RMSE: 4239618) | NHITS (RMSE: 4930875)
- Product #3 – NBEATS (RMSE: 2528315) | NHITS (RMSE: 2914800)

### Ensemble

In [ ]:
ensemble_forecasts = {}

def get_weights(rmse_dict):
    """Compute inverse RMSE weights and normalize them to sum to 1."""
    inv_rmse = {k: 1 / v for k, v in rmse_dict.items() if v > 0} 
    total = sum(inv_rmse.values())
    
    if total == 0: 
        return {k: 1 / len(rmse_dict) for k in rmse_dict}
    
    return {k: v / total for k, v in inv_rmse.items()}

arimax_weights = get_weights({gck: results[gck]['rmse'] for gck in results})
xgb_weights = get_weights(rmse_scores)
prophet_weights = get_weights({gck: prophet_results[gck]['rmse'] for gck in prophet_results})
nbeats_weights = get_weights(nbeats_rmse)
nhits_weights = get_weights(nhits_rmse)
timegpt_weights = get_weights(timegpt_rmse)

num_models = 5 

for gck in train_product_dfs.keys():
    arimax_pred = results[gck]['y_pred'].values[:len(y_val)] if gck in results else np.zeros_like(y_val)
    xgb_pred = forecasts[gck].values().flatten()[:len(y_val)] if gck in forecasts else np.zeros_like(y_val)
    prophet_pred = prophet_results[gck]['y_pred'].values[:len(y_val)] if gck in prophet_results else np.zeros_like(y_val)
    timegpt_pred = np.zeros_like(y_val)
    
    if gck in gck_predictions:
        nbeats_pred = gck_predictions[gck]['NBEATS'].values[:len(y_val)]
        nhits_pred = gck_predictions[gck]['NHITS'].values[:len(y_val)]
    else:
        nbeats_pred = np.zeros_like(y_val)
        nhits_pred = np.zeros_like(y_val)

    weights = {
        'arimax': arimax_weights.get(gck, 1 / num_models),
        'xgb': xgb_weights.get(gck, 1 / num_models),
        'prophet': prophet_weights.get(gck, 1 / num_models),
        'timegpt': timegpt_weights.get(gck, 1 / num_models),
        'nbeats': nbeats_weights.get(gck, 1 / num_models),
        'nhits': nhits_weights.get(gck, 1 / num_models),
    }

    total_weight = sum(weights.values())
    weights = {k: v / total_weight for k, v in weights.items()}

    ensemble_pred = (
        weights['arimax'] * arimax_pred +
        weights['xgb'] * xgb_pred +
        weights['prophet'] * prophet_pred +
        weights['timegpt'] * timegpt_pred +
        weights['nbeats'] * nbeats_pred +
        weights['nhits'] * nhits_pred
    )

    ensemble_forecasts[gck] = ensemble_pred

In [ ]:
unique_gcks = ensemble_forecasts.keys()

for gck in unique_gcks:
	actual_data = val_product_dfs[gck]
	forecasted_data = ensemble_forecasts[gck]
	plt.figure(figsize=(10, 6))
	sns.lineplot(x=actual_data['DATE'], y=actual_data['Sales_EUR'], label='Actual Sales', color='#009999')
	sns.lineplot(x=actual_data['DATE'], y=forecasted_data, label='Ensemble Forecast', color='#2D373C', linestyle='dashed')
	plt.title(f'Actual vs Ensemble Forecasted Sales for GCK {gck}')
	plt.xlabel('Date')
	plt.ylabel('Sales (EUR)')
	plt.legend()
	plt.grid(True)
	plt.show()

In [ ]:
ensemble_rmse = {}
for gck in unique_gcks:
    actual_sales = val_product_dfs[gck]['Sales_EUR']
    forecasted_sales = ensemble_forecasts[gck]
    ensemble_rmse[gck] = np.sqrt(mean_squared_error(actual_sales, forecasted_sales))
    print(f"RMSE for GCK {gck}: {ensemble_rmse[gck]}")

Next, the group implemented an ensemble approach combining multiple forecasting models to improve predictive accuracy.

Best Performing Models (Lowest RMSE):
- Product #20 – RMSE: 2261
- Product #9 – RMSE: 6114
- Product #13 – RMSE: 12853
- Product #14 – RMSE: 12626
- Product #36 – RMSE: 15403
- Product #16 - RMSE: 76286

Moderate Performance:
- Products such as #12 (RMSE: 106679), #11 (RMSE: 849925), #6 (RMSE: 249059), #4 (RMSE: 77273), and #8 (RMSE: 611629) showed mid-range RMSE values.

Underperforming Models (Highest RMSE):
- Product #1 – RMSE: 5935530
- Product #5 – RMSE: 4553506
- Product #3 – RMSE: 3912663

## 7. Conclusions

In this notebook, the group had several forecasting approaches, such as XGBoost, Prophet, TimeGPT, Neural Forecast (using both NBEATS and NHITS), and an Ensemble model across different product groups. The RMSE results made us conclude some points:

Strong Predictive Performance for Lower-Volume Products:
- Products such as #20, #9, #13, #14, #36 and #16 consistently had low RMSE values, regardless of the model used, suggesting that, for these products, the models predict future sales with high accuracy.

Moderate Performance for Mid-Range Products:
- Products like #12, #6, #4, #8 and #11 exhibited RMSE values in the mid-range. While the forecasts for these products are reasonably accurate, there is still room for improvement through further feature refinement or hyperparameter tuning.

Underperforming Models for High-Volume/High-Variability Products:
- Products #1, #3, and #5 produced very high RMSE values (ranging from approximately 2.8 million to over 5.9 million). These results suggest that the forecasting models struggle to capture the patterns of these product groups, which may be due to higher volatility, additional structural changes, or missing key predictive variables.

The group also concluded that, as expected, the ensemble model was the best perfomring model even though it had some obvious flaws.

## 8. Exporting the data for delivery

In [ ]:
#SARIMAX

results = {}

gck_exog_vars = {
    '#1': ['Production Index Machinery & Electricals China', 'Production Index Machinery & Electricals Switzerland', 'World: Price of Energy', 'Production Index United States: Electrical equipment', 'Production Index France: Electrical equipment'],
    '#11': ['World: Price of Metals & Minerals'],
    '#12': ['Shipments Index Machinery & Electricals France', 'Production Index Machinery & Electricals Italy', 'Production Index United States: Electrical equipment'],
    '#13': None,
    '#14': ['Production Index Machinery & Electricals Switzerland', 'World: Price of Energy'],
    '#20': ['Production Index Machinery & Electricals China', 'Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'World: Price of Copper', 'World: Price of Metals & Minerals', 'World: Price of Natural gas index', 'Production Index United Kingdom: Electrical equipment'],
    '#3': ['Production Index Machinery & Electricals United States', 'World: Price of Copper'],
    '#36': ['Production Index Machinery & Electricals United States'],
    '#4': ['Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'World: Price of Natural gas index', 'Production Index Germany: Machinery and equipment n.e.c.'],
    '#5': ['Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'Producer Prices Germany: Electrical equipment', 'Production Index United States: Electrical equipment'],
    '#6': ['Production Index Machinery & Electricals China', 'World: Price of Copper', 'World: Price of Energy'],
    '#8': ['Production Index Machinery & Electricals China', 'Shipments Index Machinery & Electricals Italy', 'Production Index Machinery & Electricals Switzerland', 'World: Price of Copper', 'Production Index Germany: Machinery and equipment n.e.c.', 'Production Index United States: Electrical equipment'],
    '#9': ['World: Price of Copper'],
    '#16' : None,
}

for gck in train_for_export.keys():
    print(f"Training ARIMAX for Product: {gck}")

    train_gck = train_for_export[gck]
    val_gck = val_for_export[gck]

    train_gck = train_gck.drop(columns=['Mapped_GCK'])
    val_gck = val_gck.drop(columns=['Mapped_GCK'])

    y_train = train_gck['Sales_EUR']
    y_val = val_gck['Sales_EUR']
    if gck_exog_vars[gck] is not None:
        X_train = train_gck[gck_exog_vars[gck]]
        X_val = val_gck[gck_exog_vars[gck]]

        model = SARIMAX(y_train, exog=X_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        model_fit = model.fit(disp=False)

        future_exog = pd.concat([X_train, X_val]).iloc[-12:]
        y_pred = model_fit.predict(start=len(y_train), end=len(y_train) + 11, exog=future_exog)
    else:
        model = ARIMA(y_train, order=(1, 1, 1))
        model_fit = model.fit()
        y_pred = model_fit.predict(start=len(y_train), end=len(y_train) + 11)

    y_pred = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_val) - 1, exog=X_val)

    y_pred.index = val_gck.index
    y_pred_rmse = y_pred[val_gck['DATE'] <= '2022-04']
    mse = mean_squared_error(y_val, y_pred_rmse)
    rmse = np.sqrt(mse)
    results[gck] = {'rmse': rmse, 'y_val': y_val, 'y_pred': y_pred_rmse}

sarimax_rmse = {gck: result['rmse'] for gck, result in results.items() if 'rmse' in result}

In [ ]:
train_for_export['#1']

In [ ]:
#XGBoost

for product in train_product_dfs.keys():
    train_product_dfs[product] = train_product_dfs[product].reset_index()
    val_product_dfs[product] = val_product_dfs[product].reset_index()

from darts import TimeSeries
from darts.models import XGBModel
from darts.metrics import rmse
from darts.models.forecasting.xgboost import XGBModel

forecasts = {}
rmse_scores = {}

for gck in train_product_dfs.keys():
    print(f"Training XGBoost for Product: {gck}")

    train_gck = train_for_export[gck].copy()
    val_gck = val_for_export[gck].copy()

    train_gck = train_gck.drop(columns=['Mapped_GCK'])
    val_gck = val_gck.drop(columns=['Mapped_GCK'])

    y_train = train_gck['Sales_EUR']
    y_val = val_gck['Sales_EUR']

    target_train = TimeSeries.from_dataframe(train_gck, time_col='DATE', value_cols=['Sales_EUR'])
    target_val = TimeSeries.from_dataframe(val_gck, time_col='DATE', value_cols=['Sales_EUR'])

    if gck in selected_features:
        X_train = train_gck[selected_features[gck]]
        X_val = val_gck[selected_features[gck]]

        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        exog_train = TimeSeries.from_dataframe(train_gck, time_col='DATE', value_cols=selected_features[gck])
        exog_val = TimeSeries.from_dataframe(val_gck, time_col='DATE', value_cols=selected_features[gck])
    else:
        exog_train, exog_val = None, None  


    model = XGBModel(lags=9, lags_exog=9 if exog_train else None)
    model.fit(target_train, past_covariates=exog_train)

    y_pred = model.predict(n=12, past_covariates=exog_val)

    forecasts[gck] = y_pred

    y_pred.index = val_gck.index
    y_pred_rmse = y_pred[val_gck['DATE'] <= '2022-04']
    rmse_value = rmse(target_val, y_pred)
    rmse_scores[gck] = rmse_value

xgb_rmse = {gck: rmse_value for gck, rmse_value in rmse_scores.items() if rmse_value is not None}

In [ ]:
# Prophet

prophet_results = {}

for product in mapped_gck_list:
    print(f"Processing Product: {product}")
    
    df_train = train_product_dfs[product].reset_index()
    df_val   = val_product_dfs[product].reset_index()
    
    df_prophet = df_train[['DATE', 'Sales_EUR']].rename(columns={'DATE': 'ds', 'Sales_EUR': 'y'})
    
    model = Prophet(seasonality_mode='multiplicative', yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    model.fit(df_prophet)
    
    forecast_periods = len(df_val)
    future = model.make_future_dataframe(periods=forecast_periods, freq='MS')
    
    forecast = model.predict(future)

    y_pred = forecast['yhat'][-forecast_periods:]

    mse = mean_squared_error(df_val['Sales_EUR'], y_pred)
    rmse = np.sqrt(mse)

    prophet_results[product] = {'rmse': rmse, 'y_val': df_val['Sales_EUR'], 'y_pred': y_pred}

    print(f"Product: {product}, RMSE: {rmse}")

    plt.figure(figsize=(12, 6))
    plt.plot(df_val['DATE'], df_val['Sales_EUR'], label='Actual Sales', color = '#009999')
    plt.plot(df_val['DATE'], y_pred, label='Predicted Sales', linestyle='dashed', color='#2D373C')
    plt.xlabel('Date')
    plt.ylabel('Sales (EUR)')
    plt.title(f'Prophet Model - Actual vs Predicted Sales for {product}')
    plt.legend()
    plt.show()

for product, result in prophet_results.items():
    print(f"Product: {product}, RMSE: {result['rmse']}")

prophet_rmse = {gck: rmse_value for gck, rmse_value in prophet_results.items() if rmse_value is not None}

In [ ]:
# TimeGPT

nixtla_client = NixtlaClient(
    api_key = 'nixak-IJDy3xU0F51O6p8oD2ZxTm1f0qOuwPofNu8bXP1NpIpGj8O5uVN8WE3YecfIJh0DLMi2AMof6672XL4t')

nixtla_client.validate_api_key()

trans_train['DATE'] = pd.to_datetime(trans_train['DATE'].dt.to_timestamp())
trans_val['DATE'] = pd.to_datetime(trans_val['DATE'].dt.to_timestamp())

timegpt_fcst_df = nixtla_client.forecast(trans_train, time_col='DATE', target_col='Sales_EUR', id_col='Mapped_GCK', h = 9, freq='MS', model='timegpt-1-long-horizon')
timegpt_fcst_df.head()

unique_gcks = timegpt_fcst_df['Mapped_GCK'].unique()

timegpt_rmse = {}
for gck in unique_gcks:
    actual_sales = trans_val[trans_val['Mapped_GCK'] == gck]['Sales_EUR']
    forecasted_sales = timegpt_fcst_df[timegpt_fcst_df['Mapped_GCK'] == gck]['TimeGPT']
    timegpt_rmse[gck] = root_mean_squared_error(actual_sales, forecasted_sales)

timegpt_rmse = {gck: rmse_value for gck, rmse_value in timegpt_rmse.items() if rmse_value is not None}
for i in timegpt_rmse:
    print(i, timegpt_rmse[i])

In [ ]:
# Neural Forecast

gck_features = {
    '#1': ['Production Index Machinery & Electricals China', 'Production Index Machinery & Electricals Switzerland', 'World: Price of Energy', 'Production Index United States: Electrical equipment', 'Production Index France: Electrical equipment'],
    '#11': ['World: Price of Metals & Minerals'],
    '#12': ['Shipments Index Machinery & Electricals France', 'Production Index Machinery & Electricals Italy', 'Production Index United States: Electrical equipment'],
    '#13': None,
    '#14': ['Production Index Machinery & Electricals Switzerland', 'World: Price of Energy'],
    '#20': ['Production Index Machinery & Electricals China', 'Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'World: Price of Copper', 'World: Price of Metals & Minerals', 'World: Price of Natural gas index', 'Production Index United Kingdom: Electrical equipment'],
    '#3': ['Production Index Machinery & Electricals United States', 'World: Price of Copper'],
    '#36': ['Production Index Machinery & Electricals United States'],
    '#4': ['Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'World: Price of Natural gas index', 'Production Index Germany: Machinery and equipment n.e.c.'],
    '#5': ['Production Index Machinery & Electricals Germany', 'Production Index Machinery & Electricals Japan', 'Producer Prices Germany: Electrical equipment', 'Production Index United States: Electrical equipment'],
    '#6': ['Production Index Machinery & Electricals China', 'World: Price of Copper', 'World: Price of Energy'],
    '#8': ['Production Index Machinery & Electricals China', 'Shipments Index Machinery & Electricals Italy', 'Production Index Machinery & Electricals Switzerland', 'World: Price of Copper', 'Production Index Germany: Machinery and equipment n.e.c.', 'Production Index United States: Electrical equipment'],
    '#9': ['World: Price of Copper'],
    '#16' : None
    }

gck_predictions = {}

for gck, features in gck_features.items():
    print(f"Processing GCK: {gck}")
    
    gck_train = train_product_dfs[gck].reset_index()
    gck_train = gck_train.rename(columns={'DATE': 'ds', 'Sales_EUR': 'y', 'Mapped_GCK': 'unique_id'})
    gck_train = gck_train[['ds', 'y', 'unique_id'] + (features if features else [])]
    
    models = [
        NBEATS(input_size=2 * 10, h=10, max_steps=100, enable_progress_bar=False),
        NHITS(input_size=2 * 10, h=10, max_steps=100, enable_progress_bar=False)
    ]
    nf = NeuralForecast(models=models, freq='MS')
    
    nf.fit(df=gck_train)
    
    Y_hat_df = nf.predict()
    gck_predictions[gck] = Y_hat_df

    plt.figure(figsize=(10, 6))
    sns.lineplot(data=val_product_dfs[gck].reset_index(), x='DATE', y='Sales_EUR', label='Actual Sales', color='#009999')
    sns.lineplot(data=Y_hat_df, x='ds', y='NBEATS', label='NBEATS', marker='x', color='#2D373C', linestyle='dashed')
    sns.lineplot(data=Y_hat_df, x='ds', y='NHITS', label='NHITS', marker='x', color='#E3700F',  linestyle='dashed')
    plt.xlabel('Date')
    plt.ylabel('Prediction')
    plt.title(f'Predicted Time Series for GCK {gck}')
    plt.legend()
    plt.show()

In [ ]:
ensemble_forecasts = {}

def get_weights(rmse_dict):
    """Compute inverse RMSE weights and normalize them to sum to 1."""
    inv_rmse = {k: 1 / v for k, v in rmse_dict.items() if v > 0} 
    total = sum(inv_rmse.values())
    
    if total == 0: 
        return {k: 1 / len(rmse_dict) for k in rmse_dict}
    
    return {k: v / total for k, v in inv_rmse.items()}

arimax_weights = get_weights({gck: results[gck]['rmse'] for gck in results})
xgb_weights = get_weights(rmse_scores)
prophet_weights = get_weights({gck: prophet_results[gck]['rmse'] for gck in prophet_results})
nbeats_weights = get_weights(nbeats_rmse)
nhits_weights = get_weights(nhits_rmse)
timegpt_weights = get_weights(timegpt_rmse)

num_models = 5 

for gck in train_product_dfs.keys():
    arimax_pred = results[gck]['y_pred'].values[:len(y_val)] if gck in results else np.zeros_like(y_val)
    xgb_pred = forecasts[gck].values().flatten()[:len(y_val)] if gck in forecasts else np.zeros_like(y_val)
    prophet_pred = prophet_results[gck]['y_pred'].values[:len(y_val)] if gck in prophet_results else np.zeros_like(y_val)
    timegpt_pred = np.zeros_like(y_val)

    if gck in gck_predictions:
        nbeats_pred = gck_predictions[gck]['NBEATS'].values[:len(y_val)]
        nhits_pred = gck_predictions[gck]['NHITS'].values[:len(y_val)]
    else:
        nbeats_pred = np.zeros_like(y_val)
        nhits_pred = np.zeros_like(y_val)

    weights = {
        'arimax': arimax_weights.get(gck, 1 / num_models),
        'xgb': xgb_weights.get(gck, 1 / num_models),
        'prophet': prophet_weights.get(gck, 1 / num_models),
        'timegpt': timegpt_weights.get(gck, 1 / num_models),
        'nbeats': nbeats_weights.get(gck, 1 / num_models),
        'nhits': nhits_weights.get(gck, 1 / num_models),
    }

    total_weight = sum(weights.values())
    weights = {k: v / total_weight for k, v in weights.items()}

    ensemble_pred = (
        weights['arimax'] * arimax_pred +
        weights['xgb'] * xgb_pred +
        weights['prophet'] * prophet_pred +
        weights['timegpt'] * timegpt_pred +
        weights['nbeats'] * nbeats_pred +
        weights['nhits'] * nhits_pred
    )

    ensemble_forecasts[gck] = ensemble_pred

In [ ]:
# Prepare the data for export
ensemble_predictions = []

for gck, predictions in ensemble_forecasts.items():
    for date, sales in zip(val_product_dfs[gck]['DATE'], predictions):
        ensemble_predictions.append({'DATE': date, 'Mapped_GCK': gck, 'Sales_EUR': sales})

# Convert to DataFrame
ensemble_predictions_df = pd.DataFrame(ensemble_predictions)

In [ ]:
# Export to CSV
ensemble_predictions_df.to_csv('ensemble_predictions.csv', index=False)

## 9. References
<sup>1</sup> Pallavi Padav. (2024, November). Granger causality in time series explained using chicken and egg problem. Analytics Vidhya. https://www.analyticsvidhya.com/blog/2021/08/granger-causality-in-time-series-explained-using-chicken-and-egg-problem/

<sup>2</sup> Adam Hayes. (2024, July). Autoregressive integrated moving average (ARIMA). Retrieved March 27, 2025, from https://www.investopedia.com/terms/a/autoregressive-integrated-moving-average-arima.asp

<sup>3</sup> GeeksforGeeks. (n.d.). Complete guide to SARIMAX in Python. GeeksforGeeks. Retrieved March 29, 2025, from https://www.geeksforgeeks.org/complete-guide-to-sarimax-in-python

<sup>4</sup> Github. (n.d. ) Prophet: Automatic Forecasting Procedure. Github. Retrieved March 29, 2025, from https://github.com/facebook/prophet

<sup>5</sup> NIXTLA. (n.d. ) About TimeGPT. NIXTLA. Retrieve March 29, 2025, from https://docs.nixtla.io/docs/getting-started-about_timegpt

<sup>6</sup> Github. (n.d. ) Neural Forecast, User friendly state-of-the-art neural forecasting models. Github. Retrieve March 29, 2025, from https://github.com/Nixtla/neuralforecast